## Imports and Defs

In [1]:
import assets.helper as b3
import assets.functions as run

from typing import Dict, Union, List, Optional, Any

import pandas as pd
import numpy as np
import plotly.express as px
import os
import plotly.io as pio
import plotly.graph_objects as go
import plotly.graph_objects as go
from plotly.graph_objs import Figure
from plotly.subplots import make_subplots

import math


## Content

#### Merge Mix and Match all

In [ ]:
try:
    math = run.load_pkl(f'{b3.app_folder}math')
except Exception as e:
    math = run.get_math()
    math = run.save_pkl(math, f'{b3.app_folder}math')


In [ ]:

try:
    setorial = run.load_pkl(f'{b3.app_folder}setorial')
except Exception as e:
    setorial = run.get_classificacao_setorial(setorial='')
    setorial = run.save_pkl(setorial, f'{b3.app_folder}setorial')
# setorial.head(3)

In [ ]:

try:
    b3_cvm = run.load_pkl(f'{b3.app_folder}b3_cvm')
except Exception as e:
    b3_cvm = run.b3_grab(b3.search_url)
    b3_cvm = run.save_pkl(b3_cvm, f'{b3.app_folder}b3_cvm')


In [ ]:
# SETOR: O setor econômico mais amplo ao qual a empresa pertence.
# SUBSETOR: Uma categorização mais específica dentro do setor mais amplo.
# SEGMENTO: Uma classificação ainda mais granular do negócio da empresa.
# DENOM_CIA: Esta é a denominação ou nome da empresa.
# COMPANHIA: Nome ou denominação oficial da empresa listada.
# PREGAO: Refere-se ao nome pelo qual a empresa é conhecida no pregão da bolsa de valores.
# LISTAGEM: Categoria ou segmento de listagem da empresa na bolsa de valores, que pode indicar o nível de governança corporativa ou outros critérios.
# TICK: Abreviação ou símbolo da empresa usada no mercado de ações.
# TICKERS: Símbolos de negociação da empresa em diferentes mercados ou plataformas.
# CD_CVM: Este poderia ser um código ou identificador único relacionado à empresa, possivelmente relacionado à Comissão de Valores Mobiliários do Brasil (CVM).
# CVM: Código ou identificador relacionado à empresa na Comissão de Valores Mobiliários, o órgão regulador do mercado de capitais no Brasil.
# ISIN: Número de Identificação Internacional de Valores Mobiliários – um identificador único para valores mobiliários.
# CNPJ_CIA: Este é o número do Cadastro Nacional da Pessoa Jurídica (CNPJ) da empresa, um identificador único para empresas no Brasil.
# CNPJ: Cadastro Nacional da Pessoa Jurídica – é o número de identificação das empresas brasileiras.
# SITE: Site oficial ou página relevante da empresa.
# ATIVIDADE: Descreve a principal atividade de negócios da empresa.

# ANO: Este é o ano ao qual os dados se referem.
# DT_REFER: Esta é a data de referência para a entrada de dados.
# DT_FIM_EXERC: Esta é a data final para o exercício ou período de relato financeiro.
# DT_INI_EXERC: Esta poderia ser a data inicial para o exercício ou período de relato financeiro.

# AGRUPAMENTO: Isso descreve o nível de agregação dos dados. Por exemplo, 'con' pode indicar dados consolidados.
# BALANCE_SHEET: Isso indica a seção específica da demonstração financeira, como Balanço Patrimonial ('BPA').
# GRUPO_DFP: Isso representa o tipo de grupo de demonstração financeira. Por exemplo, 'DF Consolidado - Balanço Patrimonial Ativo' sugere que é um balanço patrimonial consolidado focado em ativos.
# CD_CONTA: Este poderia ser um código ou identificador único relacionado a uma conta específica ou item de linha na demonstração financeira.
# DS_CONTA: Descreve a conta específica ou item de linha na demonstração financeira, como 'Ativo Total'.

# VL_CONTA: Representa o valor associado à conta específica ou item de linha.
# MOEDA: Isso indica a moeda na qual os valores são representados. 'REAL' sugere Real Brasileiro.
# ESCALA_MOEDA: Isso fornece a escala ou unidade para os valores monetários. 'MIL' pode indicar que os valores estão em milhares.

# ST_CONTA_FIXA: Pode indicar o status ou tipo de conta. O significado de valores como 'S' dependeria do contexto dos dados.
# COLUNA_DF: O propósito desta coluna não é imediatamente claro a partir da amostra. Pode representar algum tipo de classificação ou categorização relacionada aos dados financeiros.

# ESCRITURADOR: Entidade ou empresa responsável por registrar ou gerenciar os valores mobiliários da empresa.
# ACIONISTAS: Informações ou identificadores relacionados aos acionistas da empresa.

# FILENAME: Este é o arquivo de onde os dados são originados. Ele fornece o nome do arquivo que contém a respectiva entrada de dados.
# DEMONSTRATIVO: Este representa o tipo de demonstração financeira. Pode indicar se os dados são de um relatório intermediário (como 'itr') ou de outro tipo de relatório financeiro.
# VERSAO: Isso pode representar a versão ou iteração dos dados/relatórios financeiros.

columns = [
    'SETOR_x',
    'SUBSETOR_x',
    'SEGMENTO_x',
    'DENOM_CIA',
        # 'COMPANHIA',
    'PREGAO',
    'LISTAGEM',
    'TICK',
    'TICKERS',
    'CD_CVM',
        # 'CVM',
        # 'ISIN',
    'CNPJ_CIA',
        # 'CNPJ',
    'SITE',
    'ATIVIDADE',
        # 'ANO',
    'DT_REFER',
        # 'DT_FIM_EXERC',
        # 'DT_INI_EXERC',
    'AGRUPAMENTO',
    'BALANCE_SHEET',
    # 'GRUPO_DFP',
    'CD_CONTA',
    'DS_CONTA',
    'VL_CONTA',
    # 'MOEDA',
    # 'ESCALA_MOEDA',
    # 'ST_CONTA_FIXA',
    # 'COLUNA_DF',
    'ESCRITURADOR',
    'ACIONISTAS', 
    # 'FILENAME', 
    # 'DEMONSTRATIVO', 
    # 'VERSAO',
]


In [ ]:
b3_cvm.keys()

In [ ]:
columns = ['FILENAME', 'DEMONSTRATIVO', 'BALANCE_SHEET', 'ANO', 'AGRUPAMENTO',
       'CNPJ_CIA', 'DT_REFER', 'VERSAO', 'DENOM_CIA', 'CD_CVM', 'GRUPO_DFP',
       'MOEDA', 'ESCALA_MOEDA', 'DT_FIM_EXERC', 'CD_CONTA', 'DS_CONTA',
       'VL_CONTA', 'ST_CONTA_FIXA', 'DT_INI_EXERC', 'COLUNA_DF', 'COMPANHIA',
       'PREGAO', 'TICK', 'LISTAGEM', 'TICKERS', 'ISIN', 
       'ATIVIDADE', 'SETOR', 'SUBSETOR', 'SEGMENTO', 'SITE', 'ESCRITURADOR',]
df = b3_cvm['CONSUMO CICLICO'][columns].set_index('DT_REFER')
df = convert_columns(df)
df['DENOM_CIA'].unique()

#### Acoes


In [ ]:
acoes = run.load_pkl(f'{b3.app_folder}acoes')
# Process the data and return the result
acoes['Trimestre'] = pd.to_datetime(acoes['Trimestre'], errors='coerce', dayfirst=True)
acoes['BALANCE_SHEET'] = 'STK'
column_mapping = {
    'Ações ON': '00.01.01',
    'Ações PN': '00.02.01',
    'Ações ON em Tesouraria': '00.01.02',
    'Ações PN em Tesouraria': '00.02.02'
}
acoes = acoes.rename(columns={"Companhia": "DENOM_CIA", "Trimestre": "DT_REFER"})

acoes = acoes.melt(id_vars=['DENOM_CIA', 'DT_REFER', 'BALANCE_SHEET'], 
                        value_vars=['Ações ON', 'Ações PN', 'Ações ON em Tesouraria', 'Ações PN em Tesouraria'],
                        var_name='DS_CONTA', value_name='VL_CONTA').sort_values(by=['DENOM_CIA', 'DT_REFER', 'DS_CONTA'])

acoes['CD_CONTA'] = acoes['DS_CONTA'].map(column_mapping)


In [ ]:
intel_b3 = run.load_pkl(f'{b3.app_folder}intel_b3')
df = intel_b3['BENS INDUSTRIAIS']


In [ ]:
company = 'ARMAC LOCACAO LOGISTICA E SERVICOS SA'
quarter = '2019-12-31'
mc = acoes['DENOM_CIA'] == company
mc &= acoes['DT_REFER'] == quarter
acoesc = acoes[mc]

mc = df['DENOM_CIA'] == company
mc &= df['DT_REFER'] == quarter
dfc = df[mc]

df_ffill = pd.concat([dfc, acoesc], ignore_index=True).ffill()

In [ ]:
def process_stock_data(group):
    company, quarter = group.name
    
    # Filter acoes based on company and quarter
    mc = acoes['DENOM_CIA'] == company
    mc &= acoes['DT_REFER'] == quarter
    acoesc = acoes[mc]

    # Concatenate and ffill
    return pd.concat([group, acoesc], ignore_index=True).ffill()

result_df


In [ ]:

df_ffill

In [ ]:
intelacoes = {}
for setor, df in intel_b3.items():
    print(setor)
    df_concat = pd.concat([df.set_index(['DENOM_CIA', 'DT_REFER']), acoes.set_index(['DENOM_CIA', 'DT_REFER'])], axis=0, sort=False).reset_index()
    filled_df = df_concat.groupby(['DENOM_CIA', 'DT_REFER'], group_keys=False).apply(lambda group: group.ffill().bfill()).reset_index()
    intelacoes[setor] = filled_df


In [ ]:
# Concatenate them vertically
df_concat = pd.concat([df.set_index(['DENOM_CIA', 'DT_REFER']), acoes.set_index(['DENOM_CIA', 'DT_REFER'])], axis=0, sort=False).reset_index()

In [ ]:
filled_df = df_concat.groupby(['DENOM_CIA', 'DT_REFER'], group_keys=False).apply(lambda group: group.ffill().bfill()).reset_index()


In [ ]:
m = filled_df['DENOM_CIA'] == 'ARMAC LOCACAO LOGISTICA E SERVICOS SA'
m &= filled_df['DT_REFER'] == '2019-12-31'
m &= filled_df['BALANCE_SHEET'] == 'STK'

filled_df[m][cols]

#### Intel


In [ ]:
b3_cvm = run.load_pkl(f'{b3.app_folder}b3_cvm')


In [ ]:
column_types = {'index': 'int',
 'FILENAME': 'category',
 'DEMONSTRATIVO': 'category',
 'BALANCE_SHEET': 'category',
 'ANO': 'category',
 'AGRUPAMENTO': 'category',
 'CNPJ_CIA': 'category',
 'DT_REFER': 'object',
 'VERSAO': 'category',
 'DENOM_CIA': 'category',
 'CD_CVM': 'category',
 'GRUPO_DFP': 'category',
 'MOEDA': 'category',
 'ESCALA_MOEDA': 'category',
 'DT_FIM_EXERC': 'object',
 'CD_CONTA': 'category',
 'DS_CONTA': 'category',
 'VL_CONTA': 'float',
 'ST_CONTA_FIXA': 'category',
 'DT_INI_EXERC': 'object',
 'COLUNA_DF': 'category',
 'COMPANHIA': 'category',
 'PREGAO': 'category',
 'TICK': 'category',
 'LISTAGEM': 'category',
 'CVM': 'category',
 'TICKERS': 'category',
 'ISIN': 'category',
 'CNPJ': 'category',
 'ATIVIDADE': 'category',
 'SETOR': 'category',
 'SUBSETOR': 'category',
 'SEGMENTO': 'category',
 'SITE': 'category',
 'ESCRITURADOR': 'category',
 'CD_CONTA_original': 'category',
 'DS_CONTA_original': 'category'}
date_columns = ['DT_REFER', 'DT_FIM_EXERC', 'DT_INI_EXERC']


In [ ]:
df = pd.read_csv('BENS INDUSTRIAIS_intel.csv', dtype=column_types, index_col='Unnamed: 0', parse_dates=True)
# df = pd.read_csv('COMUNICACOES_df.csv')
# df_typed = pd.read_csv(f)


In [ ]:
# Criando um dicionário onde as chaves são os nomes das colunas e os valores são os valores únicos para cada coluna
col_dict = {col: df[col].unique().tolist() for col in df.columns}
df.columns

In [ ]:
col_dict['BALANCE_SHEET']

In [ ]:
df = df[['CNPJ_CIA', 'DENOM_CIA', 'DT_REFER', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA', 'SETOR', 'SUBSETOR', 'SEGMENTO', 'DT_INI_EXERC', 'ATIVIDADE', 'SITE', 'ESCRITURADOR', 'FILENAME', 'DEMONSTRATIVO', 'BALANCE_SHEET', 'ANO', 'AGRUPAMENTO', 'VERSAO', 'GRUPO_DFP', 'MOEDA', 'ESCALA_MOEDA', 'ST_CONTA_FIXA', 'COLUNA_DF', 'COMPANHIA', 'TICKERS', 'CVM', 'ISIN', 'DT_FIM_EXERC', ]]
# 'PREGAO', 'TICK', 'CD_CVM', 'LISTAGEM', # 'CD_CONTA_original', 'DS_CONTA_original', 
df

In [ ]:
formulas_old = [
    # Relações Entre Ativos e Passivos
    ('_020302_reservas_de_capital', '_020303_reservas_de_reavaliacao', '_020304_reservas_de_lucros'),
    # Dívida
    ('_0201040101_emprestimos_e_financiamentos_em_moeda_nacional', '_0201040102_emprestimos_e_financiamentos_em_moeda_estrangeira', '_02010402_debentures', '_02010403_arrendamentos', '_02010409_outros_emprestimos_financiamentos_e_debentures'),
    ('_0202010101_emprestimos_e_financiamentos_em_moeda_nacional', '_0202010102_emprestimos_e_financiamentos_em_moeda_estrangeira', '_02020102_debentures', '_02020103_arrendamentos', '_02020209_outros_emprestimos_financiamentos_e_debentures'),
    ('_0201040102_emprestimos_e_financiamentos_em_moeda_estrangeira', '_0202010102_emprestimos_e_financiamentos_em_moeda_estrangeira'),
    ('_010101_caixa_e_disponibilidades_de_caixa',),
    ('_010202_investimentos_nao_capex', '_010203_imobilizados', '_010204_intangivel'),
    ('_0305_lajir_ebit_resultado_antes_do_resultado_financeiro_e_dos_tributos', '_070401_depreciacao_e_amortizacao'),
    # Resultados Fundamentalistas
    ('_0203_patrimonio_liquido',),
    ('_010101_caixa_e_disponibilidades_de_caixa',),
    ('_070803_remuneracao_de_capital_de_terceiros', '_070804_remuneracao_de_capital_proprio'),
    # Análise do Fluxo de Caixa
    ('_0601_caixa_das_operacoes', '_0602_caixa_de_investimentos_capex'),
    ('_0603_caixa_de_financiamento',),
    ('_060201_investimentos', '_060202_imobilizado_e_intangivel'),
]

def de_transform_corrected(key):
    # Strip the leading underscore and split at the first underscore
    parts = key[1:].split('_', 1)

    # Adjust code by inserting periods every two characters
    code = '.'.join([parts[0][i:i+2] for i in range(0, len(parts[0]), 2)])
    
    # Adjust description capitalization
    description = ' '.join([word.capitalize() if word not in ['e', 'de', 'do', 'dos', 'da', 'das', 'em'] else word.lower() for word in parts[1].split('_')])
    
    return code, description

# De-transform the formulas using the corrected function
formulas = []
for group in formulas_old:
    new_group = []
    for key in group:
        new_group.append(de_transform_corrected(key))
    formulas.append(tuple(new_group))

formulas


In [ ]:
balance_sheet = df.groupby(['CNPJ_CIA', 'DT_REFER'], group_keys=True)
balance_sheet.groups.keys()
df_ = balance_sheet.get_group(('00.242.184/0001-04', '2019-12-31'))


In [ ]:
# (, 'Emprestimos e Financiamentos em Moeda Estrangeira'),
m = df['CD_CONTA'] == '07.08.04'
df[m]

##### Dados Abertos

In [ ]:
def read(file):
    path = "C:\\Users\\faust\\OneDrive\\Área de Trabalho\\dados abertos\\"
    df = pd.read_csv(path+file+".csv", sep=';', encoding='latin1')
    return df.head(25)



In [ ]:
file = "fca_cia_aberta_2010"
fca_cia_aberta_2010 = read(file)
# link para o NSD do formulário cadastral
fca_cia_aberta_2010

In [ ]:
file = "fca_cia_aberta_auditor_2010"
fca_cia_aberta_auditor_2010 = read(file)
# informações dos auditores CNPJ e CPF, datas dos auditores CPF
fca_cia_aberta_auditor_2010

In [ ]:
file = "fca_cia_aberta_canal_divulgacao_2010"
fca_cia_aberta_canal_divulgacao_2010 = read(file)
# Onde as DRE são divulgadas
fca_cia_aberta_canal_divulgacao_2010

In [ ]:
file = "fca_cia_aberta_departamento_acionistas_2010"
fca_cia_aberta_departamento_acionistas_2010 = read(file)
# Endereços dos DRI
fca_cia_aberta_departamento_acionistas_2010

In [ ]:
file = "fca_cia_aberta_dri_2010"
fca_cia_aberta_dri_2010 = read(file)
# NOMES e endereços dos DRI
fca_cia_aberta_dri_2010

In [ ]:
file = "fca_cia_aberta_endereco_2010"
fca_cia_aberta_endereco_2010 = read(file)
# Endereço completo do DRI
fca_cia_aberta_endereco_2010

In [ ]:
file = "fca_cia_aberta_geral_2010"
fca_cia_aberta_geral_2010 = read(file)
# Cadastro CVM, Atividade, Descrição e Controle Acionário
fca_cia_aberta_geral_2010

In [ ]:
file = "fca_cia_aberta_pais_estrangeiro_negociacao_2010"
fca_cia_aberta_pais_estrangeiro_negociacao_2010 = read(file)
# País estrangeiro... ?
fca_cia_aberta_pais_estrangeiro_negociacao_2010

In [ ]:
file = "fca_cia_aberta_valor_mobiliario_2010"
fca_cia_aberta_valor_mobiliario_2010 = read(file)
# Valor mobiliário, Mercado e Segmento
fca_cia_aberta_valor_mobiliario_2010

In [ ]:
file = "fre_cia_aberta_2023"
fre_cia_aberta_2023 = read(file)
# Link do Documento
fre_cia_aberta_2023

In [ ]:
file = "fre_cia_aberta_acao_entregue_2023"
fre_cia_aberta_acao_entregue_2023 = read(file)
# Remuneração da Diretoria
fre_cia_aberta_acao_entregue_2023

In [ ]:
file = "fre_cia_aberta_administrador_declaracao_genero_2023"
fre_cia_aberta_administrador_declaracao_genero_2023 = read(file)
# Gênero dos administradores
fre_cia_aberta_administrador_declaracao_genero_2023

In [ ]:
file = "fre_cia_aberta_administrador_declaracao_raca_2023"
fre_cia_aberta_administrador_declaracao_raca_2023 = read(file)
# Raça dos administradores
fre_cia_aberta_administrador_declaracao_raca_2023

In [ ]:
file = "fre_cia_aberta_administrador_membro_conselho_fiscal_2023"
fre_cia_aberta_administrador_membro_conselho_fiscal_2023 = read(file)
# Membros do Conselho Fiscal
fre_cia_aberta_administrador_membro_conselho_fiscal_2023

In [ ]:
file = "fre_cia_aberta_ativo_imobilizado_2023"
fre_cia_aberta_ativo_imobilizado_2023 = read(file)
# Ativos e Propriedades por empresa
fre_cia_aberta_ativo_imobilizado_2023

In [ ]:
file = "fre_cia_aberta_ativo_intangivel_2023"
fre_cia_aberta_ativo_intangivel_2023 = read(file)
# Ativos e Propriedades por empresa
fre_cia_aberta_ativo_intangivel_2023

In [ ]:
file = "fre_cia_aberta_auditor_2023"
fre_cia_aberta_auditor_2023 = read(file)
# Remuneração por auditor
fre_cia_aberta_auditor_2023

In [ ]:
file = "fre_cia_aberta_auditor_responsavel_2023"
fre_cia_aberta_auditor_responsavel_2023 = read(file)
# Endereço do Auditor
fre_cia_aberta_auditor_responsavel_2023

In [ ]:
file = "fre_cia_aberta_capital_social_2023"
fre_cia_aberta_capital_social_2023 = read(file)
# Modificações no Capital Social e Ações
fre_cia_aberta_capital_social_2023

In [ ]:
file = "fre_cia_aberta_capital_social_aumento_2023"
fre_cia_aberta_capital_social_aumento_2023 = read(file)
# Idem
fre_cia_aberta_capital_social_aumento_2023

In [ ]:
file = "fre_cia_aberta_capital_social_aumento_classe_acao_2023"
fre_cia_aberta_capital_social_aumento_classe_acao_2023 = read(file)
# Em branco
fre_cia_aberta_capital_social_aumento_classe_acao_2023

In [ ]:
file = "fre_cia_aberta_capital_social_classe_acao_2023"
fre_cia_aberta_capital_social_classe_acao_2023 = read(file)
# Preferencial Classe A, B e C
fre_cia_aberta_capital_social_classe_acao_2023

In [ ]:
file = "fre_cia_aberta_capital_social_desdobramento_2023"
fre_cia_aberta_capital_social_desdobramento_2023 = read(file)
# Desdobramentos de Ações
fre_cia_aberta_capital_social_desdobramento_2023

In [ ]:
file = "fre_cia_aberta_capital_social_desdobramento_classe_acao_2023"
fre_cia_aberta_capital_social_desdobramento_classe_acao_2023 = read(file)
# em branco
fre_cia_aberta_capital_social_desdobramento_classe_acao_2023

In [ ]:
file = "fre_cia_aberta_capital_social_reducao_2023"
fre_cia_aberta_capital_social_reducao_2023 = read(file)
# Redução de capital
fre_cia_aberta_capital_social_reducao_2023

In [ ]:
file = "fre_cia_aberta_capital_social_titulo_conversivel_2023"
fre_cia_aberta_capital_social_titulo_conversivel_2023 = read(file)
# Redução de capital
fre_cia_aberta_capital_social_titulo_conversivel_2023

In [ ]:
file = "fre_cia_aberta_direito_acao_2023"
fre_cia_aberta_direito_acao_2023 = read(file)
# ?
fre_cia_aberta_direito_acao_2023

In [ ]:
file = "fre_cia_aberta_historico_emissor_2023"
fre_cia_aberta_historico_emissor_2023 = read(file)
# Constituição do emissor e local
fre_cia_aberta_historico_emissor_2023

In [ ]:
file = "fre_cia_aberta_informacao_financeira_2023"
fre_cia_aberta_informacao_financeira_2023 = read(file)
# DRE Resumido
fre_cia_aberta_informacao_financeira_2023

In [ ]:
file = "fre_cia_aberta_membro_comite_2023"
fre_cia_aberta_membro_comite_2023 = read(file)
# CPF e Remuneração
fre_cia_aberta_membro_comite_2023

In [ ]:
file = "fre_cia_aberta_obrigacao_2023"
fre_cia_aberta_obrigacao_2023 = read(file)
# Obrigações e Dívidas
fre_cia_aberta_obrigacao_2023

In [ ]:
file = "fre_cia_aberta_outro_valor_mobiliario_2023"
fre_cia_aberta_outro_valor_mobiliario_2023 = read(file)
# ?
fre_cia_aberta_outro_valor_mobiliario_2023

In [ ]:
file = "fre_cia_aberta_participacao_sociedade_2023"
fre_cia_aberta_participacao_sociedade_2023 = read(file)
# Participações em outras empresas
fre_cia_aberta_participacao_sociedade_2023

In [ ]:
file = "fre_cia_aberta_participacao_sociedade_valorizacao_acao_2023"
fre_cia_aberta_participacao_sociedade_valorizacao_acao_2023 = read(file)
# ?
fre_cia_aberta_participacao_sociedade_valorizacao_acao_2023

In [ ]:
file = "fre_cia_aberta_posicao_acionaria_2023"
fre_cia_aberta_posicao_acionaria_2023 = read(file)
# Composição acionária por acionista majoritário
fre_cia_aberta_posicao_acionaria_2023[['CNPJ_Companhia', 'Data_Referencia', 'Versao', 'ID_Documento',
       'ID_Acionista', 'Acionista', 'Tipo_Pessoa_Acionista',
       'CPF_CNPJ_Acionista', 'ID_Acionista_Relacionado',
       'Acionista_Relacionado', 'Tipo_Pessoa_Acionista_Relacionado',
       'CPF_CNPJ_Acionista_Relacionado',
       'Quantidade_Acao_Ordinaria_Circulacao',
       'Percentual_Acao_Ordinaria_Circulacao',
       'Quantidade_Acao_Preferencial_Circulacao',
       'Percentual_Acao_Preferencial_Circulacao',
       'Quantidade_Total_Acoes_Circulacao',
       'Percentual_Total_Acoes_Circulacao', ]]
# fre_cia_aberta_posicao_acionaria_2023.columns


In [ ]:
file = "fre_cia_aberta_relacao_familiar_2023"
fre_cia_aberta_relacao_familiar_2023 = read(file)
# Parentescos
fre_cia_aberta_relacao_familiar_2023

In [ ]:
file = "fre_cia_aberta_relacao_subordinacao_2023"
fre_cia_aberta_relacao_subordinacao_2023 = read(file)
# Subordnicação 
fre_cia_aberta_relacao_subordinacao_2023

In [ ]:
file = "fre_cia_aberta_transacao_parte_relacionada_2023"
fre_cia_aberta_transacao_parte_relacionada_2023 = read(file)
# Partes relacionadas
fre_cia_aberta_transacao_parte_relacionada_2023

In [ ]:
file = "fre_cia_aberta_2023"
fre_cia_aberta_2023 = read(file)
# Link do Documento
fre_cia_aberta_2023

In [ ]:
mudança commit só no main branch. E com atualização

In [ ]:
def plot_group(cias_por_setor, window):
    for company, group_df in cias_por_setor:
        try:
            # Calculate the moving average for the last 4 periods
            group_df['MA'] = group_df['VL_CONTA'].rolling(window=window).mean()
            
            # Calculate the rolling sum for the last 4 periods
            group_df['Rolling_Sum'] = group_df['VL_CONTA'].rolling(window=window).sum()
            
            # Calculate the lifelong cumulative sum
            group_df['Cumulative_Sum'] = group_df['VL_CONTA'].cumsum()
            
            # Plot raw data
            # group_df['VL_CONTA'].plot(label='Raw Data', legend=True)

            # Plot moving average
            group_df['MA'].plot(label=f'{window} Quarters Moving Average', legend=True, linestyle='--')
            
            # Plot rolling sum
            group_df['Rolling_Sum'].plot(label=f'{window} Quarters Sum', legend=True, linestyle='-.')
            
            # Plot lifelong cumulative sum
            group_df['Cumulative_Sum'].plot(label='Lifelong Cumulative Sum', legend=True, linestyle='-.')

            plt.title(f"{group_df['CD_CVM'].iloc[-1]} {group_df['DENOM_CIA'].iloc[-1]}")
            plt.show()
        except Exception as e:
            print(f"Error plotting for {company}: {e}")
    return True

cias_por_setor = df[(df['AGRUPAMENTO'] == 'con') & (df['CD_CONTA'] == '3.11')].groupby('CD_CVM')
plot_group(cias_por_setor, window)

In [ ]:
import matplotlib.pyplot as plt

cias_por_setor = df[(df['AGRUPAMENTO'] == 'con') & (df['CD_CONTA'] == '2.03')].groupby('DENOM_CIA')

fig, ax = plt.subplots(figsize=(10, 6))

for company, group_df in cias_por_setor:
    try:
        group_df[['VL_CONTA']].plot(ax=ax, label=company)
    except:
        print(company)

ax.set_title("VL_CONTA by Company")
ax.set_ylabel("VL_CONTA")
ax.set_xlabel("Index")
ax.legend(loc="best")

plt.show()


#### Intelacoes Fundamentalista

In [ ]:
intelacoes = run.load_pkl(f'{b3.app_folder}intelacoes')
intelacoes.keys()

In [ ]:
df = pd.read_pickle('BENS INDUSTRIAIS_intelacoes.pkl')
# df.to_csv('BENS INDUSTRIAIS_intelacoes.csv')

In [ ]:
columns = ['Unnamed: 0', 'FILENAME', 'DEMONSTRATIVO', 'BALANCE_SHEET', 'ANO',
       'AGRUPAMENTO', 'CNPJ_CIA', 'DT_REFER', 'VERSAO', 'DENOM_CIA', 'CD_CVM',
       'GRUPO_DFP', 'MOEDA', 'ESCALA_MOEDA', 'DT_FIM_EXERC', 'CD_CONTA',
       'DS_CONTA', 'VL_CONTA', 'ST_CONTA_FIXA', 'DT_INI_EXERC', 'COLUNA_DF',
       'COMPANHIA', 'PREGAO', 'TICK', 'LISTAGEM', 'CVM', 'TICKERS', 'ISIN',
       'CNPJ', 'ATIVIDADE', 'SETOR', 'SUBSETOR', 'SEGMENTO', 'SITE',
       'ESCRITURADOR', 'CD_CONTA_original', 'DS_CONTA_original', 'Companhia',
       'Trimestre', 'Ações ON', 'Ações PN', 'Ações ON em Tesouraria',
       'Ações PN em Tesouraria', 'URL']
cols = ['SETOR', 'SUBSETOR', 'SEGMENTO', 'CNPJ_CIA', 'DENOM_CIA', 'CD_CVM',  'PREGAO', 'TICK', 'LISTAGEM', 'TICKERS', 'DT_REFER', 'BALANCE_SHEET', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA', ]
datetime_cols = ['DT_REFER', 'DT_FIM_EXERC', 'DT_INI_EXERC', 'Trimestre']
company_cols = ['SETOR', 'SUBSETOR', 'SEGMENTO', 'CNPJ_CIA', 'DENOM_CIA', 'CD_CVM']
dateseries_col = ['DT_REFER']
sheet_cols = ['BALANCE_SHEET', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA']
stocks_cols = ['Ações ON', 'Ações PN', 'Ações ON em Tesouraria', 'Ações PN em Tesouraria']


In [ ]:
# Grouping by 'SETOR', 'SUBSETOR', and 'SEGMENTO' to count unique companies and aggregate their TICK values
ticks_by_setor = df.groupby(['SETOR', 'SUBSETOR', 'SEGMENTO']).agg({
    'DENOM_CIA': 'nunique',
    'TICK': lambda x: list(set(x.dropna()))
}).reset_index()

# Renaming columns for clarity
ticks_by_setor.rename(columns={'DENOM_CIA': 'TOTAL DE COMPANHIAS'}, inplace=True)

ticks_by_setor


In [ ]:
conta = '02.03'

# Filtering the data for the account 'CD_CONTA' with value '02.03' and the latest quarter
conta_by_setor = df[df['DT_REFER'] == df['DT_REFER'].max()]
conta_by_setor = conta_by_setor[conta_by_setor['CD_CONTA'] == conta]

# Grouping by 'SETOR', 'SUBSETOR', and 'SEGMENTO' to aggregate 'VL_CONTA' values
conta_by_setor = conta_by_setor.groupby(['SETOR', 'SUBSETOR', 'SEGMENTO', ]).agg({
    'VL_CONTA': ['nunique', 'sum', 'max', 'min', 'mean', 'std', 'skew', lambda x: x.kurt()]
}).reset_index()

conta_by_setor.rename(columns={'VL_CONTA': conta}, inplace=True)

conta_by_setor


In [ ]:
dfc = df
# dfc = dfc.set_index('DT_REFER')
company = 'WEG SA'
quarter = '2020-12-31'
conta = '01.01.01'

In [ ]:
mask = dfc['DENOM_CIA'] != 'nothing'
mask &= dfc['DT_REFER'] == quarter
mask &= dfc['DENOM_CIA'] == company
# mask &= dfc['PREGAO'] == ''
# mask &= dfc['TICK'] == ''
# mask &= dfc['BALANCE_SHEET'] == ''
mask &= dfc['CD_CONTA'] == conta
# mask &= dfc['DS_CONTA'] == ''

# dfc[mask][cols]
data = dfc[mask][['DENOM_CIA', 'DT_REFER', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA']].set_index('DT_REFER')
sheet = df[(df['DENOM_CIA'] == company) & (df['DT_REFER'] == quarter)]
data

In [ ]:
dfc = pd.concat([dfc, rows], ignore_index=True).ffill().drop_duplicates()
m = dfc['BALANCE_SHEET'].isin(sh)
dfc[m][cols]

#### Fund

In [ ]:
fund = load_pkl(f'{b3.app_folder}fund')


In [ ]:
df = pd.read_pickle('BENS INDUSTRIAIS_fund.pkl')

In [ ]:
df.to_csv('BENS INDUSTRIAIS_fund.csv')

In [ ]:
df.columns

In [ ]:
df['TICKERS'].unique().tolist()

#### Macro Data

In [ ]:
!pip install yfinance -q -U

In [ ]:
import yfinance as yf

In [ ]:
# Baixando Cotações do Índice Bovespa
ibov = yf.download('^BVSP')
df = yf.download(['WEGE3.SA','BBDC4.SA', 'PETR4.SA'], group_by='ticker')

In [ ]:
print(type(df['WEGE3.SA']['Adj Close']))
tickers = df.columns.get_level_values(0).unique().tolist()
tickers

In [ ]:
for tick in tickers:
    df[tick]['Adj Close'].plot()

In [ ]:
!pip install -q alpha_vantage
# Importando a classe Timeseries de alpha_vantage.timeseries
from alpha_vantage.timeseries import TimeSeries

In [ ]:
ALPHAVANTAGE_API_KEY = 'KR3OMFL1CLANZUXP'
# Criando o objeto ts
ts = TimeSeries(key=ALPHAVANTAGE_API_KEY, output_format='pandas')

In [ ]:
dados, meta_dados = ts.get_symbol_search('alphabet')
dados

In [ ]:
# Obtendo os dados semanais do IBOV usando get_weekly
dados, meta_dados = ts.get_daily(symbol='AAPL', )
dados['4. close']

In [ ]:
meta_dados

In [ ]:
! pip install quandl -q -U

In [ ]:
import quandl
mydata = quandl.get("FRED/GDP")
api = 'LpAz8JCUosdwhHqnWnA4'

In [ ]:
mydata = quandl.get_table('ZACKS/FC', ticker='AAPL',)
mydata

In [ ]:
mydata.plot()

In [ ]:
# pandas datareader
import os
import pandas_datareader as pdr


In [ ]:
TIINGO_API_KEY = '591375a6e5852ca48f778902fec322581511c89a'
import tiingo

config = {}
config['session'] = True
config['api_key'] = TIINGO_API_KEY
client = tiingo.TiingoClient(config)



In [ ]:
import requests

TIINGO_API_KEY = '591375a6e5852ca48f778902fec322581511c89a'
url = 'https://api.tiingo.com/tiingo/fundamentals/definitions'
url = 'https://api.tiingo.com/tiingo/daily/AAPL/prices?startDate=2012-1-1'
# url = 'https://api.tiingo.com/tiingo/daily/AAPL/prices'

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Token ' + TIINGO_API_KEY
}

response = requests.get(url, headers=headers)

data = response.json()
data = pd.DataFrame(data)


In [ ]:
data['close'].plot()

In [ ]:
df = pdr.DataReader('GE', 'yahoo')
df

#### Get Yahoo CSV

iterrow in fund and get same key from both dict to concat both and remove duplicates

save quotes

#### Get BCB Series

In [ ]:
from selenium.webdriver.common.alert import Alert
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [ ]:
chromedriver_path = "D:\\Fausto Stangler\\Documentos\\Python\\DSH\\chromedriver-win64\\chromedriver.exe"
driver_wait_time = 5

driver, wait = run.load_browser(chromedriver_path, driver_wait_time)


In [ ]:
url_bcb = 'https://www3.bcb.gov.br/sgspub/'
url_bcb = 'https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaPesquisaAvancada'

driver.get(url_bcb)

try:
    alert = Alert(driver)
    alert.accept()
except Exception as e:
    pass


In [ ]:
element_xpath = '/html/body/table[1]/tbody/tr/td[3]/a'
element = wait.until(EC.presence_of_element_located((By.XPATH, element_xpath)))
# Get elements in Português
if 'Português' in element.text.splitlines():
    element.click()
# # Get elements in English
# if 'English' in element.text.splitlines():
#     element.click()

# Click on the first element
element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tabLCS"]/tbody/tr/td[1]/table/tbody/tr[12]/td')))
element.click()

# Switch to the iframe
iframe_id = 'iCorpo'
wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, iframe_id)))

# Click on the second element
element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/center/form/table[1]/tbody/tr[5]/td[2]/input')))
element.click()

# Click on the third element
element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/center/form/input[9]')))
element.click()

# Grab the number of items for pagination
pagination_xpath = '/html/body/form/span[1]/span[1]/b'
num_items = int(wait.until(EC.presence_of_element_located((By.XPATH, pagination_xpath))).text)

# Read the table
table_xpath = '//*[@id="tabelaSeries"]'  # or '//*[@id="tabelaConjunto"]'
table_html = wait.until(EC.presence_of_element_located((By.XPATH, table_xpath))).get_attribute('outerHTML')
df = pd.read_html(table_html)[0]

# Determine the number of pages for pagination (assuming 10 items per page)
items_per_page = len(df)  # Adjust as per actual items per page
num_pages = -(-num_items // items_per_page)  # Ceiling division
items_per_page, num_pages


In [ ]:
# Loop through the pages and extract data
start_time = run.time.time()
for i, page_number in enumerate(range(num_pages - 1)):  # Already have data for first page
        # Execute the JavaScript function for pagination
    driver.execute_script(f'getPagina({page_number})')
    
    # Read the table on the new page
    table_html = wait.until(EC.presence_of_element_located((By.XPATH, table_xpath))).get_attribute('outerHTML')
    new_df = pd.read_html(table_html)[0]
    
    # Loop through each row of the new data
    for _, row in new_df.iterrows():
        # Extract the relevant parameter for JavaScript function from the row
        series = row['Cód.']  # replace 'YourColumnName' with the actual column name
        
        # Execute the JavaScript function
        driver.execute_script(f"parent.pesquisarPorDocn('../localizarseries/localizarSeries.do?method=recuperarMetadadosPorDocn', '{series}', 'Dados básicos/Metadados');")
        print('grab the table xpath and save into new_df')
        # Grab the data you need after executing the JavaScript
        # Your code for grabbing data goes here...
        
        # NOTE: Be sure to implement appropriate waiting and checking for elements to ensure data is loaded before you try to grab it.

    # Append the new data to the existing dataframe
    df = pd.concat([df, new_df], ignore_index=True)
    print(run.remaining_time(start_time, (num_pages - 1), i))

In [ ]:
new_df

In [ ]:
# now get each series data

#### Integrate fund to quotes

Integrate funds to quotes, so each TICK contains quotes and also also all financial data

In [ ]:
quotes = run.load_pkl(f'{b3.app_folder}quotes')
fund = run.load_pkl(f'{b3.app_folder}fund')


In [ ]:
quotes.keys(), quotes['AERIS'].keys()

In [ ]:
bigdata = []
for pregao, d in quotes.items():
    for ticker, df in d.items():
        try:
            df.insert(0, 'PREGAO', pregao)
        except Exception as e:
            df['PREGAO'] = pregao  # Update 'PREGAO' column
        
        try:
            df['TICKER'] = ticker  # Update 'TICKER' column
        except Exception as e:
            df.insert(1, 'TICKER', ticker)

        bigdata.append(df)
bigdata = pd.concat(bigdata, ignore_index=False)
bigdata


In [ ]:
fund.keys()


In [ ]:
def add_metrics(df):
    # Ensure no division by zero for all columns
    df.replace(0, np.nan, inplace=True)



    return df


In [ ]:
df_plitly = {}

start_time = run.time.time()
for i, (setor, df_fund) in enumerate(fund.items()):
    # Data preprocessing
    # Convert specific columns to object type
    df_fund[['VERSAO', 'CD_CVM']] = df_fund[['VERSAO', 'CD_CVM']].astype('object')

    # Convert 'DT_REFER' to datetime format
    df_fund['DT_REFER'] = pd.to_datetime(df_fund['DT_REFER'])

    # Pivot for CD_CONTA
    df_cd_conta = df_fund.pivot_table(index=['DT_REFER', 'PREGAO'], 
                                    columns=['CD_CONTA', 'DS_CONTA'], 
                                    values='VL_CONTA', 
                                    aggfunc='sum').reset_index()

    # Flatten the multi-level columns after pivot
    df_cd_conta.columns = [' - '.join(col).strip(' - ') for col in df_cd_conta.columns.values]

    # Extract unique combinations of DT_REFER and PREGAO without the account details and get an unique mapping between the dates, PREGAO, and the pivoted account data.
    df_unique = df_fund.reset_index(drop=True).drop_duplicates(subset=['DT_REFER', 'PREGAO']).drop(['CD_CONTA', 'DS_CONTA', 'VL_CONTA'], axis=1)
    df_merged = pd.merge(df_unique, df_cd_conta, on=['DT_REFER', 'PREGAO'])

    # Set index and handle missing values
    df_merged = df_merged.set_index(['DT_REFER', 'PREGAO'], drop=True)

    # Group by 'PREGAO' and apply resampling
    df_resampled = (
        df_merged
        .reset_index()
        .groupby('PREGAO')
        .apply(lambda group: group.set_index('DT_REFER').resample('D').asfreq().ffill().bfill().fillna(0))
        .drop('PREGAO', axis=1)  # Drop the redundant 'PREGAO' column introduced by `groupby`
    )

    try:
        bigdata = bigdata.reset_index()
    except Exception as e:
        pass
    bigdata['Date'] = pd.to_datetime(bigdata['Date'])
    
    try:
        df_resampled = df_resampled.reset_index()
    except Exception as e:
        pass
    df_resampled['DT_REFER'] = pd.to_datetime(df_resampled['DT_REFER'])

    # Step 1: Get unique PREGAO values from df_resampled
    companies = df_resampled['PREGAO'].unique()

    # Step 2: Filter bigdata
    filtered_bigdata = bigdata[bigdata['PREGAO'].isin(companies)]

    df_merged = pd.merge(filtered_bigdata, df_resampled, left_on=['Date', 'PREGAO'], right_on=['DT_REFER', 'PREGAO'], how='outer')

    # Sort the dataframe by 'PREGAO' and 'Date'
    df_merged = df_merged.sort_values(by=['PREGAO', 'Date'])

    # Group by 'PREGAO', then apply the fill methods within each group
    df_merged = df_merged.groupby('PREGAO', group_keys=False).apply(lambda group: group.ffill().bfill()).fillna(0).reset_index(drop=True)

    # Fill any remaining NaN values in the entire dataframe (outside of groups)
    df_merged = df_merged.set_index('Date', drop=True)

    df_merged = df_merged.groupby('PREGAO', group_keys=False).apply(add_metrics)

    df_plitly[setor] = df_merged

    print(setor, run.remaining_time(start_time, len(fund), i))


In [3]:
# Define the path to the folder
company_folder = 'company'
folder_path = os.path.join(b3.app_folder, company_folder)

# Check if the folder does not exist
if not os.path.exists(folder_path):
    # Create the folder
    os.makedirs(folder_path)


In [ ]:
failed = []
start_time = run.time.time()
for i, (setor, df) in enumerate(df_plitly.items()):
    companies = df['PREGAO'].unique()
    for i2, company in enumerate(companies):
        mask = df['PREGAO'] == company
        df_temp = df[mask]
        try:
            df_temp = run.save_pkl(df_temp, f'{b3.app_folder}{company_folder}/{company}')
        except Exception as e:
            failed.append(company)
    print(setor, run.remaining_time(start_time, len(df_plitly), i))
failed

In [4]:
df = run.load_pkl(f'{b3.app_folder}{company_folder}/MAGAZ LUIZA')
df.columns.to_list()

['level_0',
 'index',
 'PREGAO',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'TICKER',
 'DT_REFER',
 'FILENAME',
 'DEMONSTRATIVO',
 'BALANCE_SHEET',
 'ANO',
 'AGRUPAMENTO',
 'CNPJ_CIA',
 'VERSAO',
 'DENOM_CIA',
 'CD_CVM',
 'GRUPO_DFP',
 'MOEDA',
 'ESCALA_MOEDA',
 'DT_FIM_EXERC',
 'ST_CONTA_FIXA',
 'DT_INI_EXERC',
 'COLUNA_DF',
 'COMPANHIA',
 'TICK',
 'LISTAGEM',
 'CVM',
 'TICKERS',
 'ISIN',
 'CNPJ',
 'ATIVIDADE',
 'SETOR',
 'SUBSETOR',
 'SEGMENTO',
 'SITE',
 'ESCRITURADOR',
 'CD_CONTA_original',
 'DS_CONTA_original',
 '00.01.01 - Ações ON',
 '00.01.02 - Ações ON em Tesouraria',
 '00.02.01 - Ações PN',
 '00.02.02 - Ações PN em Tesouraria',
 '01 - Ativo Total',
 '01.01 - Ativo Circulante de Curto Prazo',
 '01.01.01 - Caixa e Disponibilidades de Caixa',
 '01.01.02 - Aplicações Financeiras',
 '01.01.03 - Contas a Receber',
 '01.01.04 - Estoques',
 '01.01.05 - Ativos Biológicos',
 '01.01.06 - Tributos',
 '01.01.07 - Despesas',
 '01.01.09 - Outros Ativos Circulantes',
 '01.

Os dados estão disponíveis de 31/12/2010 até 30/06/2023


In [ ]:
companies

In [ ]:
company = companies[0]
company = 'WEG'
df = df_merged[df_merged['PREGAO'] == company]


In [ ]:
setor, subsetor, segmento = df[['SETOR', 'SUBSETOR', 'SEGMENTO']].iloc[0]
setor, subsetor, segmento

In [ ]:
m_cia = df_merged['PREGAO'] != company
m_segmento = df_merged['SEGMENTO'] == segmento
m_subsetor = df_merged['SUBSETOR'] == subsetor
m_setor = df_merged['SETOR'] == setor

cias_segmento = [cia for cia in df_merged[m_cia & m_segmento]['PREGAO'].unique().tolist()]
cias_subsetor = [cia for cia in df_merged[m_cia & m_subsetor]['PREGAO'].unique().tolist() if cia not in cias_segmento]
cias_setor = [cia for cia in df_merged[m_cia & m_setor]['PREGAO'].unique().tolist() if cia not in cias_subsetor and cia not in cias_segmento]
cias_segmento, cias_subsetor, cias_setor


In [ ]:
tickers = np.sort(df['TICKER'].unique())
df_ticker = []
for ticker in tickers:
    df_ticker.append(df[df['TICKER'] == ticker])
df = df_ticker[0]

In [ ]:
df_merged.to_csv('dash_df_merged.csv')
df.to_csv('dash_df.csv')

In [ ]:
df_merged = run.save_pkl(df_merged, 'dash_df_merged')
df = run.save_pkl(df, 'dash_df')


#### Plotly

In [ ]:
line = '13.09.02 - Outros Ativos Não Circulantes de Longo Prazo por Faturamento'
data = {
    'title': [f'{item} - {plot}', 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [item], 
    'right': [cagr(item, years), ofs(item, years)], 
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, 'mma': [years, 2], },
    'right': {'shape': 'ine', 'mode': 'standalone', 'normalization': False, 'outliers': False, }, 
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company][item][plot] = fig
fig.show()


In [ ]:
def plot_dual_axis(df, data):
    '''
    data: Dicionário contendo chaves 'left' e 'right', com as colunas correspondentes.
               Exemplo: {'left': [col1, col2], 'right': [col3, col4]}
    df: DataFrame contendo os dados.
    '''
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Determine y-axis ranges
    factor = 1.50
    left_min = df[data['left']].min().min() if df[data['left']].min().min() < 0 else df[data['left']].min().min() * -1
    left_min *= factor
    left_max_abs = max(abs(df[data['left']].min().min()), abs(df[data['left']].max().max())) * factor

    right_min = df[data['right']].min().min() if df[data['right']].min().min() < 0 else df[data['right']].min().min() * -1
    right_min *= factor
    right_max_abs = max(abs(df[data['right']].min().min()), abs(df[data['right']].max().max())) * factor

    # Find the zero alignment factor
    zero_factor = abs(left_min) / left_max_abs

    # Adjust the right y-axis ranges to align the zeros
    right_min_adj = -right_max_abs * zero_factor
    right_max_adj = right_max_abs * (1 - zero_factor)

    # Adicione os dados do eixo esquerdo
    for column in data['right']:
        fig.add_trace(
            go.Scatter(x=df.index, y=df[column], mode='lines', name=column.split(' - ')[1]),
            secondary_y=True
        )

    # Adicione os dados do eixo direito
    fill_mode = 'tozeroy'
    for column in data['left']:
        fig.add_trace(
            go.Scatter(x=df.index, y=df[column], fill=fill_mode, name=column.split(' - ')[1]),
            secondary_y=False
        )
        fill_mode = 'tonexty'

    fig.update_layout(
        title_text=f'{company} - {data["title"][0]}',
        xaxis_title='Data',
        yaxis_title=data['title'][1],
        yaxis2_title=data['title'][2],
        yaxis_range=[left_min, left_max_abs],
        yaxis2_range=[right_min_adj, right_max_adj]
    )

    return fig


In [ ]:
def plot_stacked_area(df, data):
    '''
    df: DataFrame containing the data.
    data: Dictionary containing the columns to be plotted.
    '''
    fig = go.Figure()

    # Helper function to get data from column name or series
    def get_data(item):
        if isinstance(item, str):
            return df[item]
        return item

    # For 'left' data
    for item in data['left']:
        data_values = get_data(item)
        name = item.split(' - ')[1] if isinstance(item, str) else item.name
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=data_values,
                name=name,
                fill='tonexty', 
                stackgroup='left',
                # legendgroup='left',
                showlegend=True
            )
        )

    # For 'right' data
    for item in data['right']:
        data_values = get_data(item)
        name = item.split(' - ')[1] if isinstance(item, str) else item.name
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=data_values,
                name=name,
                fill='tonexty', 
                stackgroup='right',
                # legendgroup='right',
                showlegend=True,
                yaxis='y2'
            )
        )

    fig.update_layout(
        title_text=f'{company} - {data["title"][0]}',
        xaxis_title='Data',
        yaxis_title=data['title'][1],
        yaxis2=dict(title=data['title'][2], overlaying='y', side='right')
    )

    return fig


In [ ]:
def plot_100_stacked_area(df, data):
    '''
    df: DataFrame containing the data.
    data: Dictionary containing the columns to be plotted.
    '''
    fig = go.Figure()

    # Helper function to get data from column name or series
    def get_data(item):
        if isinstance(item, str):
            return df[item]
        return item

    # Function to normalize data
    def normalize_data(columns):
        temp_df = df[columns].copy()
        temp_df['total'] = temp_df.sum(axis=1)
        for column in columns:
            temp_df[column] = temp_df.apply(lambda row: row[column] / row['total'] * 100 if row['total'] != 0 else 0, axis=1)
        return temp_df

    left_normalized = normalize_data(data['left'])
    right_normalized = normalize_data(data['right'])

    # Plot 'left' data
    for column in data['left']:
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=left_normalized[column],
                name=column.split(' - ')[1],
                fill='tonexty',
                stackgroup='left',
                legendgroup='left',
            )
        )

    # Plot 'right' data on secondary y-axis
    for column in data['right']:
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=right_normalized[column],
                name=column.split(' - ')[1],
                fill='tonexty',
                stackgroup='right',
                legendgroup='right',
                yaxis='y2'
            )
        )

    fig.update_layout(
        title_text=f'{company} - {data["title"][0]}',
        xaxis_title='Data',
        yaxis_title=data['title'][1],
        yaxis2=dict(title=data['title'][2], overlaying='y', side='right')
    )

    return fig


In [ ]:
def plot_lines(df, data):
    '''
    data: Dicionário contendo chaves 'left' e 'title'.
               Exemplo: {'left': [col1, col2], 'title': 'Equity Multiplier'}
    df: DataFrame contendo os dados.
    '''
    fig = go.Figure()

    # Adicione os dados do eixo esquerdo
    for column in data['left']:
        fig.add_trace(
            go.Scatter(x=df.index, y=df[column], fill='none', name=column.split(' - ')[1])
        )

    fig.update_layout(
        title_text=f'{company} - {data["title"][0]}',
        xaxis_title='Data',
        yaxis_title=data['title'][1],
    )

    return fig

In [ ]:
def plot_basics(df, col, window):
    figs = {}
    if df[col].sum() != 0:

        year_average = df[col].rolling(window=365).mean()

        # Calculate rolling mean and standard deviation
        rolling_average = df[col].rolling(window=window).mean()
        rolling_std = df[col].rolling(window=window).std()

        # Create a single line plot
        title = f'A - Linha do Tempo'
        fig_line = px.line(df, x=df.index, y=col, title=f'{company} - {col} - {title}',
                            labels={'value': 'Valor (R$)', 'variable': f'{title}'})
        fig_line.add_scatter(x=df.index, y=year_average, mode='lines', line=dict(color='blue'), name='Média Anual')
        figs[title] = fig_line
        # fig_line.show()

        # Create a moving average plot with ±2 standard deviations
        title = f'B - Média Móvel e ±2 Desvios Padrão'
        fig_mma_std = px.area(df, x=df.index, y=col, title=f'{company} - {col} - {title}',
                            labels={'value': 'Valor (R$)', 'variable': f'{title}'})
        fig_mma_std.add_scatter(x=df.index, y=year_average, mode='lines', line=dict(color='blue'), name='Média Anual')
        fig_mma_std.add_scatter(x=df.index, y=rolling_average, mode='lines', line=dict(color='green'), name='Média Móvel')
        fig_mma_std.add_scatter(x=df.index, y=rolling_average + 2 * rolling_std, mode='lines', line=dict(color='green', dash='dash'), name='Média Móvel + 2 Desvios Padrão')
        fig_mma_std.add_scatter(x=df.index, y=rolling_average - 2 * rolling_std, mode='lines', line=dict(color='green', dash='dash'), name='Média Móvel - 2 Desvios Padrão')
        figs[title] = fig_mma_std
        # fig_mma_std.show()

    sub_cols = [c for c in df.columns if c.startswith(col.split(' - ')[0] + '.') and c.count('.') == col.count('.') + 1]
    if sub_cols:
        # Create a multi line plot
        title =  f'C - Distribuição Individual'
        fig_multiline = px.line(df, x=df.index, y=sub_cols, title=f'{company} - {col} - {title}',
                            labels={'value': 'Valor (R$)', 'variable': f'{title}'})
        figs[title] = fig_multiline
        # fig_area.show()

        # Create a cumulative distribution plot
        title =  f'D - Distribuição Acumulada'
        fig_area = px.area(df, x=df.index, y=sub_cols, title=f'{company} - {col} - {title}',
                            labels={'value': 'Valor (R$)', 'variable': f'{title}'})
        figs[title] = fig_area
        # fig_area.show()

        # Create a proportional distribution plot
        title = f'E - Distribuição Proporcional'
        fig_area_100_stacked = px.area(df, x=df.index, y=sub_cols, title=f'{company} - {col} - {title}',
                            labels={'value': 'Porcentagem (%)', 'variable': f'{title}'},
                            groupnorm='percent')
        figs[title] = fig_area_100_stacked
        # fig_area_100_stacked.show()

    return figs


In [ ]:
def save_figs(figs, base_dir='./company/'):
    """
    Save figures to the specified directory.

    Parameters:
    - col_figs: Dictionary of figures to save
    - col: Column for which figures were generated
    - company: Company for which figures were generated
    - base_dir: Base directory where the figures will be saved
    """
    for company, col in figs.items():
        path = os.path.join(base_dir, company)
        
        # Create company directory if it doesn't exist
        if not os.path.exists(path):
            os.makedirs(path)

        for col, plots in col.items():
            for title, plot in plots.items():
                try:
                    file_name = f'{company} - {col} - {title}.png'
                    file_path = os.path.join(path, file_name)
                    plot.write_image(file_path)
                    # plot.show()
                    print(file_name)
                except Exception as e:
                    print(e)
    return figs


In [ ]:
years = 3
figs = {}
figs[company] = {}
df = df_ticker[0]
for item in df.columns:
    figs[company][item] = {}
    print(item)
    try:
        if df[item].sum() != 0:
            plot = 'Valores, Média, CAGR e OFS'
            data = {
                'title': [f'{item} - {plot}', 'Reais (R$)', 'Porcentagem (%)'], 
                'left': [item], 
                'right': [cagr(item, years), ofs(item, years)], 
            }
            options = {
                'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, 'mma': [years, 2], },
                'right': {'shape': 'ine', 'mode': 'standalone', 'normalization': False, 'outliers': False, }, 
            }
            fig = plot_tweak(df_ticker[0], data, options)
            figs[company][item][plot] = fig
            # fig.show()

            sub_cols = [column for column in df.columns if column.startswith(item.split(' - ')[0] + '.') and column.count('.') == item.count('.') + 1]
            if sub_cols:
                plot = 'Composição Cumulativa'
                data = {
                    'title': [f'{item} - {plot}', 'Reais (R$)', 'Porcentagem (%)'], 
                    'left': sub_cols, 
                    'right': [], 
                }
                options = {
                    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, },
                    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, }, 
                }
                fig = plot_tweak(df_ticker[0], data, options)
                figs[company][item][plot] = fig
                # fig.show()

                plot = 'Composição Individual'
                data = {
                    'title': [f'{item} - {plot}', 'Reais (R$)', 'Porcentagem (%)'], 
                    'left': sub_cols, 
                    'right': [], 
                }
                options = {
                    'left': {'shape': 'line', 'mode': 'standalone', 'normalization': False, },
                    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, }, 
                }
                fig = plot_tweak(df_ticker[0], data, options)
                figs[company][item][plot] = fig
                # fig.show()

                plot = 'Composição Relativa'
                data = {
                    'title': [f'{item} - {plot}', 'Reais (R$)', 'Porcentagem (%)'], 
                    'left': sub_cols, 
                    'right': [], 
                }
                options = {
                    'left': {'shape': 'line', 'mode': 'standalone', 'normalization': True, },
                    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, }, 
                }
                fig = plot_tweak(df_ticker[0], data, options)
                figs[company][item][plot] = fig
                # fig.show()

    except Exception as e:
        pass


In [ ]:
lines = {
    'Equity Multiplier': {
        'data': {
            'title': ['Reais (R$)', 'Porcentagem (%)'],
            'left': ['01 - Ativo Total', '02.03 - Patrimônio Líquido'],
            'right': ['11.03.01 - Equity Multiplier (Ativos por Patrimônio Líquido)']
        },
        'options': {
            'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': True, 'range': 'flexible'},
            'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False}
        }
    },
    'Proporção dos Ativos': {
        'data': {
            'title': ['Reais (R$)', 'Porcentagem (%)'],
            'left': ['11.01.03 - Ativos Circulantes de Curto Prazo por Ativos', '11.01.04 - Ativos Não Circulantes de Longo Prazo por Ativos'],
            'right': []
        },
        'options': {
            'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': True, 'range': 'flexible'},
            'right': {'shape': 'line', 'mode': 'standalone', 'normalization': True}
        }
    }
}


In [ ]:
lines = {}
lines['Proporção dos Ativos'] = {'data': {'title': ['Reais (R$)', 'Porcentagem (%)'], 'left': ['11.01.03 - Ativos Circulantes de Curto Prazo por Ativos', '11.01.04 - Ativos Não Circulantes de Longo Prazo por Ativos'], 'right': []}, 'options': {'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': True, 'range': 'flexible'}, 'right': {'shape': 'line', 'mode': 'standalone', 'normalization': True}}}
lines['Proporção dos Ativos']['data']['title']

In [ ]:
title = 'Equity Multiplier'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['01 - Ativo Total', '02.03 - Patrimônio Líquido'], 
    'right': [ '11.03.01 - Equity Multiplier (Ativos por Patrimônio Líquido)',], 
}
options = {
    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': True, 'range': 'flexible', },
    'right': {'shape': 'ine', 'mode': 'standalone', 'normalization': False, }, 
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Proporção dos Ativos'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [ '11.01.03 - Ativos Circulantes de Curto Prazo por Ativos', '11.01.04 - Ativos Não Circulantes de Longo Prazo por Ativos',], 
    'right': [], 
}
options = {
    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': True, 'range': 'flexible', },
    'right': {'shape': 'ine', 'mode': 'standalone', 'normalization': True, }, 
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Proporção dos Passivos'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [ '11.02.01 - Passivos Circulantes de Curto Prazo por Ativos', '11.02.02 - Passivos Não Circulantes de Longo Prazo por Ativos', '11.03 - Patrimônio Líquido por Ativos'], 
    'right': [], 
}
options = {
    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': True, 'range': 'flexible', },
    'right': {'shape': 'ine', 'mode': 'standalone', 'normalization': False, }, 
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Passivos por Patrimônio Líquido'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [ '11.03.02.01 - Passivos Circulantes de Curto Prazo por Patrimônio Líquido',  '11.03.02.02 - Passivos Não Circulantes de Longo Prazo por Patrimônio Líquido',], 
    'right': [], 
}
options = {
    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': True, 'range': 'flexible', },
    'right': {'shape': 'ine', 'mode': 'standalone', 'normalization': False, 'outliers': True, }, 
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Ativos e Liquidez'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['01.01 - Ativo Circulante de Curto Prazo', '02.01 - Passivo Circulante de Curto Prazo'], 
    'right': ['11.01.02 - Liquidez (Ativos Circulantes por Passivos Circulantes)', ], 
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'right': {'shape': 'ine', 'mode': 'standalone', 'normalization': False, 'mma': [3,2]}, 
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Ativos e Prazos'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [df['01.01 - Ativo Circulante de Curto Prazo'], df['01.02 - Ativo Não Circulante de Longo Prazo']], 
    'right': ['11.01.03 - Ativos Circulantes de Curto Prazo por Ativos', '11.01.04 - Ativos Não Circulantes de Longo Prazo por Ativos', ], 
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': True, },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Capital de Giro'
trace = (df['01.02 - Ativo Não Circulante de Longo Prazo']/df['02.02 - Passivo Não Circulante de Longo Prazo'])
trace.name = 'Liquide de Longo Prazo (Ativos Não Circulantes por Passivos Não Circulantes)'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['11.01.02 - Liquidez (Ativos Circulantes por Passivos Circulantes)', trace], 
    'right': ['11.01.01 - Capital de Giro (Ativos Circulantes - Passivos Circulantes)'], 
}
options = {
    'left': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'range': 'flexible', },
    'right': {'shape': 'area', 'mode': 'standalone', 'normalization': False, 'range': 'flexible', },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Contas a Receber e Faturamento'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['01.01.03 - Contas a Receber', '01.02.01.03 - Contas a Receber'], 
    'right': ['13.03 - Contas por Faturamento'], 
}
options = {
    'left': {'shape': 'line', 'mode': 'standalone', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3,2], },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Estoques e Faturamento'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['01.01.04 - Estoques', '01.02.01.04 - Estoques'], 
    'right': ['13.04 - Estoques por Faturamento'], 
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, 'range': 'flexible', },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3,2], 'range': 'flexible', },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Ativos Biológicos e Faturamento'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['01.01.05 - Ativos Biológicos', '01.02.01.05 - Ativos Biológicos'], 
    'right': ['13.05 - Ativos Biológicos por Faturamento'], 
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3,2]},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Despesas e Faturamento'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['01.01.07 - Despesas', '01.02.01.07 - Despesas'], 
    'right': ['13.07 - Despesas por Faturamento'], 
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3,2]},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Tributos e Faturamento'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['01.01.06 - Tributos', '01.02.01.06 - Tributos'], 
    'right': ['13.06 - Tributos por Faturamento'], 
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3,2]},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Capex Ativos Imobilizados e Intangíveis'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['01.02.03 - Imobilizados', '01.02.04 - Intangível', '01.02.02 - Investimentos Não Capex'], 
    'right': ['01.02.03 - Imobilizados', '01.02.04 - Intangível', '01.02.02 - Investimentos Não Capex'], 
}
options = {
    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': True, },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Imobilização do Patrimônio'
trace = (df['11.03.04 - Patrimônio Imobilizado']/df['02.03 - Patrimônio Líquido'])
trace.name = 'Patrimônio Imobilizado por Patrimônio Líquido'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['11.03.04 - Patrimônio Imobilizado', '02.03 - Patrimônio Líquido'], 
    'right': [trace], 
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'cumulative', 'normalization': False, 'mma': [3,2]},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Prazo da Dívida'
trace = df['12.01.02.01 - Dívida Bruta Circulante de Curto Prazo']/df['12.01.02 - Dívida Bruta']
trace.name = 'Dívida de Curto Prazo por Dívida '
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['12.01.02.01 - Dívida Bruta Circulante de Curto Prazo', '12.01.02.02 - Dívida Bruta Não Circulante de Longo Prazo Prazo'], 
    'right': [trace], 
}
options = {
    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False,'mma': [3,2]},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Jurisdição da Dívida'
trace = df['12.01.02.03 - Dívida Bruta em Moeda Nacional']/(df['12.01.02.03 - Dívida Bruta em Moeda Nacional']+df['12.01.02.04 - Dívida Bruta em Moeda Estrangeira'])
trace.name = 'Nacionalização da Dívida'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['12.01.02.03 - Dívida Bruta em Moeda Nacional', '12.01.02.04 - Dívida Bruta em Moeda Estrangeira'], 
    'right': [trace], 
}
options = {
    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False,'mma': [3,2], },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Dívida de Curto Prazo por Ativo de Curto Prazo'
trace = df['12.01.02.01 - Dívida Bruta Circulante de Curto Prazo']/df['01.01 - Ativo Circulante de Curto Prazo']
trace.name = 'Dívida de Curto Prazo por Ativo de Curto Prazo'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['02.01.04.01.01 - Empréstimos e Financiamentos em Moeda Nacional', '02.01.04.01.02 - Empréstimos e Financiamentos em Moeda Estrangeira', '02.01.04.02 - Debêntures', '02.01.04.03 - Arrendamentos', '02.01.04.09 - Outros empréstimos, financiamentos e debêntures', ], 
    'left_01': ['01.01 - Ativo Circulante de Curto Prazo'], 
    'right': [trace], 
}
options = {
    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, },
    'left_01': {'shape': 'line', 'mode': 'standalone', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2]},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Dívida de Longo Prazo por Ativo de Longo Prazo'
trace = df['12.01.02.02 - Dívida Bruta Não Circulante de Longo Prazo Prazo']/df['01.02 - Ativo Não Circulante de Longo Prazo']
trace.name = 'Dívida de Longo Prazo por Ativo de Longo Prazo'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['02.02.01.01.01 - Empréstimos e Financiamentos em Moeda Nacional', '02.02.01.01.02 - Empréstimos e Financiamentos em Moeda Estrangeira', '02.02.01.02 - Debêntures', '02.02.01.03 - Arrendamentos', '02.02.02.09 - Outros empréstimos, financiamentos e debêntures', ], 
    'left_01': ['01.02 - Ativo Não Circulante de Longo Prazo'], 
    'right': [trace], 
}
options = {
    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, 'range': 'flexible'},
    'left_01': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'range': 'flexible'},
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], 'range': 'flexible'},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Dívida por Patrimônio Líquido'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['12.01.02.01 - Dívida Bruta Circulante de Curto Prazo', '12.01.02.02 - Dívida Bruta Não Circulante de Longo Prazo Prazo', ], 
    'left_01': ['02.03 - Patrimônio Líquido'], 
    'right': ['12.02.01 - Dívida Bruta por Patrimônio Líquido'], 
}
options = {
    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, },
    'left_01': {'shape': 'line', 'mode': 'standalone', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3,2], },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Dívida Líquida'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [df['12.01.02 - Dívida Bruta'], ], 
    'left_01': [df['01.01.01 - Caixa e Disponibilidades de Caixa']*-1], 
    'left_02': [df['12.01.03 - Dívida Líquida']], 
}
options = {
    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, 'range': 'flexible'},
    'left_01': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, 'range': 'flexible'},
    'left_02': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'range': 'flexible'},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Dívida Líquida por EBITDA'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [df['12.04.01 - Dívida Líquida por EBITDA']*-1, ], 
    'left_01': ['12.02.01 - Dívida Bruta por Patrimônio Líquido'],
    'right': ['12.01.02 - Dívida Bruta', '02.03 - Patrimônio Líquido'], 
    # 'right': [ofs('12.04.01 - Dívida Líquida por EBITDA', 3), ]
}
options = {
    'left': {'shape': 'line', 'mode': 'cumulative', 'normalization': False, 'range': 'half'},
    'left_01': {'shape': 'line', 'mode': 'cumulative', 'normalization': False, 'range': 'half'},
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'outliers': False, 'range': 'flexible'},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Endividamento Financeiro'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [df['02.03 - Patrimônio Líquido'], df['12.01.02 - Dívida Bruta'], ], 
    'right': ['12.02.02 - Endividamento Financeiro'], 
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'cumulative', 'normalization': False, 'mma': [3, 2]},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Serviço da Dívida'
trace = df['12.01.03 - Dívida Líquida']/df['03.11 - Lucro Líquido']
trace.name = 'Serviço da Dívida'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['12.01.03 - Dívida Líquida', '03.11 - Lucro Líquido'], 
    'right': [trace], 
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Índice de Cobertura'
trace = df['07.08.03.01 - Juros Pagos']/df['03.05 - LAJIR EBIT Resultado Antes do Resultado Financeiro e dos Tributos']
trace.name = 'Juros por EBIT'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['07.08.03.01 - Juros Pagos', '03.05 - LAJIR EBIT Resultado Antes do Resultado Financeiro e dos Tributos', ], 
    'right': [trace], 
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Composição do Patrimônio Líquido'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['02.03.01 - Capital Social', '02.03.02 - Reservas de Capital', '02.03.03 - Reservas de Reavaliação', '02.03.04 - Reservas de Lucros', '02.03.05 - Lucros ou Prejuízos Acumulados', '02.03.06 - Ajustes de Avaliação Patrimonial', '02.03.07 - Ajustes Acumulados de Conversão', '02.03.08 - Outros Resultados Abrangentes', ], 
}
options = {
    'left': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, 'range': 'flexible', },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Composição do Patrimônio Líquido'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': ['02.03.01 - Capital Social', '02.03.02 - Reservas de Capital', '02.03.03 - Reservas de Reavaliação', '02.03.04 - Reservas de Lucros', '02.03.05 - Lucros ou Prejuízos Acumulados', '02.03.06 - Ajustes de Avaliação Patrimonial', '02.03.07 - Ajustes Acumulados de Conversão', '02.03.08 - Outros Resultados Abrangentes', ], 
}
options = {
    'left': {'shape': 'line', 'mode': 'standalone', 'normalization': False, },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()



In [ ]:
title = 'Margem Bruta'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left_1': [df['03.02 - Custo de Produção']*-1, df['03.03 - Resultado Bruto (Receita Líquida)'], ], 
    'right': [df['16.01 - Margem Bruta (Resultado Bruto (Receita Líquida) por Receita Bruto)'] * 100, ]
}
options = {
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()



In [ ]:
title = 'Margem Operacional'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left_1': [df['03.01 - Receita Bruta'], df['03.04 - Despesas Operacionais']*-1, ], 
    'right': [df['16.02 - Margem Operacional (Receitas Operacionais por Receita Bruta)'] * -100,]
}
options = {
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, 'range': 'flexible', },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()



In [ ]:
title = 'Margem EBIT'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left_1': [df['03.04 - Despesas Operacionais']*-1, df['03.05 - LAJIR EBIT Resultado Antes do Resultado Financeiro e dos Tributos'], ], 
    'right': [df['16.03.01 - Margem EBIT (EBIT por Resultado Bruto (Receita Líquida)'] * 100,]
}
options = {
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False,},
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], 'range': 'flexible', },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()



In [ ]:
title = 'Margem de Depreciação'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left_1': [df['03.01 - Receita Bruta'], df['07.04.01 - Depreciação e Amortização'] * -1, ], 
    'right': [df['16.03.02 - Margem de Depreciação por Resultado Bruto (Receita Líquida)'] * -100,]
}
options = {
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False,},
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], 'range': 'flexible', },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()



In [ ]:
title = 'Margem EBITDA'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left_1': [df['03.05 - LAJIR EBIT Resultado Antes do Resultado Financeiro e dos Tributos'], df['13.01 - LAJIDA EBITDA Resultado Antes do Resultado Financeiro e dos Tributos mais Depreciação e Amortização'], df['07.04.01 - Depreciação e Amortização'] * -1], 
    'right': [df['16.03 - Margem EBITDA (EBITDA por Resultado Bruto (Receita Líquida)'] * 100, df['16.03.02 - Margem de Depreciação por Resultado Bruto (Receita Líquida)'] * 100, ]
}
options = {
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False,},
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], 'range': 'flexible', },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()




In [ ]:
title = 'Resultado Financeiro'
trace = df['03.06 - Resultado Financeiro (Não Operacional)']/df['03.05 - LAJIR EBIT Resultado Antes do Resultado Financeiro e dos Tributos']
trace.name = 'Margem Financeira'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left_1': [df['03.05 - LAJIR EBIT Resultado Antes do Resultado Financeiro e dos Tributos'], df['03.06 - Resultado Financeiro (Não Operacional)']], 
    'right': [trace * 100, ]
}
options = {
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False,},
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], 'range': 'flexible', },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()




In [ ]:
title = 'Margem Líquida'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [df['03.01 - Receita Bruta']], 
    'left_1': [df['03.02 - Custo de Produção'] * -1, df['03.04 - Despesas Operacionais'] * -1, df['03.06 - Resultado Financeiro (Não Operacional)'], df['03.08 - Impostos IRPJ e CSLL'] * -1, df['03.11 - Lucro Líquido']], 
    'right': [df['16.05 - Margem Líquida (Lucro Líquido por Receita Bruta)'] * 100, ]
}
options = {
    'left': {'shape': 'line', 'mode': 'standalone', 'normalization': False,'range': 'flexible', },
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False,'range': 'flexible', },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], 'range': 'half', },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()




In [ ]:
title = 'Resultado e Margem Líquida'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [df['03.01 - Receita Bruta']], 
    'left_1': [df['03.11 - Lucro Líquido']], 
    'right': [df['16.05 - Margem Líquida (Lucro Líquido por Receita Bruta)'] * 100, ]
}
options = {
    'left': {'shape': 'line', 'mode': 'standalone', 'normalization': False,'range': 'flexible', },
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False,'range': 'flexible', },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], 'range': 'half', },
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()




In [ ]:
title = 'Patrimônio e Resultado (ROE)'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [df['02.03 - Patrimônio Líquido']], 
    'left_1': [df['03.11 - Lucro Líquido']], 
    'right': [df['14.04.01 - ROE (Resultado por Patrimônio)'] * 100, ]
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], 'range': 'flexible'},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()




In [ ]:
title = 'Capital Investido e Resultado (ROIC)'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [df['14.03 - Capital Investido']], 
    'left_1': [df['03.05 - LAJIR EBIT Resultado Antes do Resultado Financeiro e dos Tributos']], 
    'right': [df['14.03.01 - ROIC (Retorno por Capital Investido)'] * 100, ]
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], 'range': 'flexible'},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()




In [ ]:
title = 'Ativos e Resultado (ROAS)'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [df['01 - Ativo Total']], 
    'left_1': [df['03.05 - LAJIR EBIT Resultado Antes do Resultado Financeiro e dos Tributos']], 
    'right': [df['14.05.01 - ROAS (EBIT por Ativos)'], ]
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], 'range': 'flexible'},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()




In [ ]:
title = 'Ativos e Resultado (Coeficiente de Retorno)'
trace = df['03.11 - Lucro Líquido'] / df['01 - Ativo Total']
trace.name = 'Coeficiente de Retorno'
data = {
    'title': [title, 'Reais (R$)', 'Porcentagem (%)'], 
    'left': [df['01 - Ativo Total']], 
    'left_1': [df['03.11 - Lucro Líquido']], 
    'right': [trace * 100, ]
}
options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, },
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'mma': [3, 2], 'range': 'flexible'},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'ROE, ROIC, ROAS e Coeficiente de Retorno'
trace = df['03.11 - Lucro Líquido'] / df['01 - Ativo Total']
trace.name = 'Coeficiente de Retorno'
data = {
    'title': [title, 'Porcentagem (%)', 'Reais (R$)', ], 
    # 'left': [df['01 - Ativo Total']], 
    # 'left_1': [df['03.11 - Lucro Líquido']], 
    'right': [df['14.04.01 - ROE (Resultado por Patrimônio)'] * 100, df['14.03.01 - ROIC (Retorno por Capital Investido)'] * 100, df['14.05.01 - ROAS (EBIT por Ativos)'] * 100, trace * 100, ]
}

options = {
    'left': {'shape': 'area', 'mode': 'standalone', 'normalization': False, 'range': 'half'},
    'left_1': {'shape': 'area', 'mode': 'cumulative', 'normalization': False, },
    'right': {'shape': 'line', 'mode': 'standalone', 'normalization': False, 'range': 'half'},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
title = 'Price'
if df.TICKER[0][-1] == '3':
    acoes = df['00.01.01 - Ações ON']
else:
    acoes = df['00.02.01 - Ações PN']

trace = df['Adj Close'] / df['00.01.01 - Ações ON']
trace.name = 'Preço por Ação'
data = {
    'title': [title, 'Porcentagem (%)', 'Reais (R$)', ], 
    'left': [df['Adj Close']], 
    'left_1': [df['03.11 - Lucro Líquido']], 
    'left_2': [acoes], 
}
options = {
    'left': {'shape': 'line', 'mode': 'standalone', 'normalization': True, 'range': 'flexible'},
    'left_1': {'shape': 'line', 'mode': 'standalone', 'normalization': True, 'range': 'flexible'},
    'left_2': {'shape': 'line', 'mode': 'standalone', 'normalization': True, 'range': 'flexible'},
}
fig = plot_tweak(df_ticker[0], data, options)
# figs[company]['others'][title] = fig
fig.show()


In [ ]:
figs = save_figs(figs)

In [ ]:
def my_decorator(func):
    print("Code outside the wrapper, this is run once during decoration")
    
    def wrapper(*args, **kwargs):
        print("Code inside the wrapper, this is run every time the decorated function is called")
        return func(*args, **kwargs)
    
    return wrapper

@my_decorator
def my_function():
    print("This is the decorated function")

print("Calling my_function")
my_function()
print("Calling my_function again")
my_function()


## DASHBOARD

In [ ]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import ThemeSwitchAIO
from dash import dash_table

import pandas as pd


In [ ]:
df_merged = run.load_pkl('dash_df_merged')
df = run.load_pkl('dash_df')

In [ ]:
quotes = run.load_pkl(f'{b3.app_folder}quotes')
fund = run.load_pkl(f'{b3.app_folder}fund')


In [ ]:
quotes.keys(), fund.keys()

In [ ]:
df_fund = pd.concat(fund.values(), ignore_index=True)


In [ ]:
quotes = run.load_pkl(f'{b3.app_folder}quotes')
fund = run.load_pkl(f'{b3.app_folder}fund')
df_fund = pd.concat(fund.values(), ignore_index=True)
df_fund = run.save_pkl(df_fund, f'{b3.app_folder}df_fund')


In [ ]:
# df_fund = run.load_pkl(f'{b3.app_folder}df_fund')
df_fund = run.load_pkl(f'df_fund')


In [ ]:
df_fund.info()

In [ ]:
df_fund['SETOR'].unique()
mask_setor = df_fund['SETOR'] == 'BENS INDUSTRIAIS'
df_fund[mask_setor]['SUBSETOR'].unique()

In [ ]:
company = df_merged['PREGAO'].iloc[0]
company = 'WEG'
setor, subsetor, segmento = df[['SETOR', 'SUBSETOR', 'SEGMENTO']].iloc[0]

m_cia = df_merged['PREGAO'] != company
m_segmento = df_merged['SEGMENTO'] == segmento
m_subsetor = df_merged['SUBSETOR'] == subsetor
m_setor = df_merged['SETOR'] == setor

cias_segmento = [cia for cia in df_merged[m_cia & m_segmento]['PREGAO'].unique().tolist()]
cias_subsetor = [cia for cia in df_merged[m_cia & m_subsetor]['PREGAO'].unique().tolist() if cia not in cias_segmento]
cias_setor = [cia for cia in df_merged[m_cia & m_setor]['PREGAO'].unique().tolist() if cia not in cias_subsetor and cia not in cias_segmento]

setor, subsetor, segmento

In [ ]:
print(df['SETOR'].unique())


In [ ]:
awsome = ["https://use.fontawesome.com/releases/v5.10.2/css/all.css"]
# app = dash.Dash(__name__, external_stylesheets=awsome)
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, awsome])
app2 = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, awsome])

# Initialize setup
tab_card = {'height': '100%'}
main_config = {
    'hover_mode': 'x unified', 
    'legend': 
        {
        'yanchor': 'top',
        'y': 0.9, 
        'xanchor': 'left', 
        'x': 0.1, 
        'title': {'text': None, }, 
        'font': {'color': 'white', }, 
        'bgcolor': 'rgba(0,0,0,0.5)', 
        }, 
    'margin': {'l':10, 'r':10, 't':10, 'b':10}, 
}
graph_config = {
    'displayModeBar': False, 
    'showTips': False, 
}
template_theme_1 = 'flatly'
template_theme_2 = 'darkly'
url_theme_1 = dbc.themes.FLATLY
url_theme_2 = dbc.themes.DARKLY


In [ ]:
# Define the layout of the app
app.layout = dbc.Container(children=[
    dbc.Row([ # Row 1
        dbc.Col([ # Row 1, Col 1
            dbc.Card([
                dbc.CardBody([
                    dbc.Row([ # Row 1, Col 1, Card 1 / Row 1
                        dbc.Col([ # Row 1, Col 1, Card 1 / Row 1, Col 1
                            html.Legend('Sobre')
                        ], sm=8), 
                        dbc.Col([ # Row 1, Col 1, Card 1 / Row 1, Col 2
                            html.I(className='fa fa-balance-scale', style={'font-size': '300%'})
                        ] , sm=4, align='center'), 
                    ], style={'margin-top': '10px', }), 
                    dbc.Row([ # Row 1, Col 1, Card 1 / Row 2
                        dbc.Col([ # Row 1, Col 1, Card 1 / Row 2, Col 1
                            ThemeSwitchAIO(aio_id='theme', themes=[url_theme_1, url_theme_2]), 
                            html.Legend('MLGS'), 
                        ]), 
                    ], style={'margin-top': '10px', }), 
                    dbc.Row([
                        dbc.Button('Site da B3', href='https://www.google.com', target='b3')
                    ]), 
                ], style=tab_card), 
            ]), 
        ], sm=12, lg=3), 
        dbc.Col([ # Row 1, Col 2
            dbc.Card([
                dbc.CardBody([
                    dbc.Row([ # Row 1, Col 1, Card 2 / Row 2
                        html.Legend(f'Companhia {company}'), 
                    ]), 
                    dbc.Row([ # Row 1, Col 1, Card 2 / Row 1
                        dbc.Col([# Row 1, Col 1, Card 2 / Row 1, Col 1
                            html.Legend(f'Setor {setor}: Companhias {", ".join(cias_setor)}'), 
                        ]), 
                        dbc.Col([# Row 1, Col 1, Card 2 / Row 1, Col 2
                            html.Legend(f'SubSetor {subsetor}: Companhias {", ".join(cias_subsetor)}'), 
                        ]), 
                        dbc.Col([# Row 1, Col 1, Card 2 / Row 1, Col 3
                            html.Legend(f'Segmento {segmento}: Companhias {", ".join(cias_segmento)}'), 
                        ]), 
                    ]), 
                ]), 
            ]), 
        ]), 
    ], className='g-2 my-auto', style={'matgin-top': '7px'}, ), 
], fluid=True, style={'height': '100vh'})

# Run the app
app.run_server(mode='external')



In [ ]:
# App layout
app.layout = dbc.Container([
    # Row 1
    dbc.Row([
        # Row 1, Col 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    dbc.Row([ # Card Row 1
                        dbc.Row(html.H2('Análise Fundamentalista')),
                        dbc.Col(html.I(className='fa fa-balance-scale', style={'font-size': '300%'}), width=4, align='center'),
                    ], style={'margin-top': '10px'}),
                    dbc.Row([ # Card Row 2
                        dbc.Col([
                            ThemeSwitchAIO(aio_id='theme', themes=[url_theme_1, url_theme_2]), 
                        ]),
                    ], style={'margin-top': '10px'}),
                    dbc.Row([
                        dbc.Col([
                            dcc.Dropdown(
                                id='setor-dropdown',
                                options=[{'label': i, 'value': i} for i in df_fund['SETOR'].unique()],
                                value=None,  # No default value
                                multi=False,
                                placeholder="Select a SETOR"
                            ),
                        ]),
                        dbc.Col([
                            dcc.Dropdown(
                                id='subsetor-dropdown',
                                multi=False,
                                placeholder="Select a SUBSETOR"
                            ),
                        ]),
                        dbc.Col([
                            dcc.Dropdown(
                                id='segmento-dropdown',
                                multi=False,
                                placeholder="Select a SEGMENTO"
                            ),
                        ]),
                    ]),
                ]),
            ),
            width=3  # 1/4 of the width
        ),
        # Row 1, Col 2
        dbc.Col(
            [
                # Row 1, Col 2 / Row 1
                dbc.Row(html.H2(f'{company} {df["LISTAGEM"].iloc[0]}')),
                # Row 1, Col 2 / Row 2
                dbc.Row([
                    html.P([
                        f"{df['ATIVIDADE'].iloc[0]}",
                    ]), 
                    html.P([
                        f"{df['ESCRITURADOR'].iloc[0]}", " / ", 
                        html.A(f"{df['CNPJ'].iloc[0]}", href=f"https://www.google.com/search?q={df['CNPJ'].iloc[0]}", target='_blank'), 
                    ]), 
                ]),
                # Row 2
                dbc.Row([
                    dbc.Col([
                        html.Span('Setor'), html.Br(),
                        html.Strong(f'{setor}'), html.Br(),
                        html.P(f'{", ".join(cias_setor)}')
                    ]),
                    dbc.Col([
                        html.Span('SubSetor'), html.Br(),
                        html.Strong(f'{subsetor}'), html.Br(),
                        html.P(f'{", ".join(cias_subsetor)}')
                    ]),
                    dbc.Col([
                        html.Span('Segmento'), html.Br(),
                        html.Strong(f'{segmento}'), html.Br(),
                        html.P(f'{", ".join(cias_segmento)}')
                    ]),
                ]),
            ],
            width=9  # 3/4 of the width
        ),
    ]),

    # Row with 1 column
    dbc.Row([
        # Column 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    # Table to display companies
                    dbc.Row([
                        dbc.Col([
                            dash_table.DataTable(
                                id='company-table',
                                columns=[
                                    {"name": "PREGAO", "id": "PREGAO"},
                                    {"name": "TICKER", "id": "TICKER"}
                                ]
                            )
                        ])
                    ])
                ])
            ),
            width=12  # Full width
        ),
    ], style={'margin-top': '20px'}),




    # Row with 1 column
    dbc.Row([
        # Column 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 1', className='card-title'),
                    html.P('Content for column 1...', className='card-text')
                ])
            ),
            width=12  # Full width
        ),
    ], style={'margin-top': '20px'}),

    # Row with 2 columns
    dbc.Row([
        # Column 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 1', className='card-title'),
                    html.P('Content for column 1...', className='card-text')
                ])
            ),
            width=6  # Half of the width
        ),
        # Column 2
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 2', className='card-title'),
                    html.P('Content for column 2...', className='card-text')
                ])
            ),
            width=6  # Half of the width
        ),
    ], style={'margin-top': '20px'}),

    # Row with 3 columns
    dbc.Row([
        # Row 2, Col 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 1', className='card-title'),
                    html.P('Content for column 1...', className='card-text')
                ])
            ),
            width=4  # 1/3 of the width
        ),
        # Row 2, Col 2
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 2', className='card-title'),
                    html.P('Content for column 2...', className='card-text')
                ])
            ),
            width=4  # 1/3 of the width
        ),
        # Row 2, Col 3
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 3', className='card-title'),
                    html.P('Content for column 3...', className='card-text')
                ])
            ),
            width=4  # 1/3 of the width
        ),
    ], style={'margin-top': '20px'}),  # Added margin for better visual separation of the rows

    # Row with 4 columns
    dbc.Row([
        # Column 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 1', className='card-title'),
                    html.P('Content for column 1...', className='card-text')
                ])
            ),
            width=3  # 1/4 of the width
        ),
        # Column 2
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 2', className='card-title'),
                    html.P('Content for column 2...', className='card-text')
                ])
            ),
            width=3  # 1/4 of the width
        ),
        # Column 3
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 3', className='card-title'),
                    html.P('Content for column 3...', className='card-text')
                ])
            ),
            width=3  # 1/4 of the width
        ),
        # Column 4
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 4', className='card-title'),
                    html.P('Content for column 4...', className='card-text')
                ])
            ),
            width=3  # 1/4 of the width
        ),
    ], style={'margin-top': '20px'}),

    # Row with 5 columns
    dbc.Row([
        # Column 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 1', className='card-title'),
                    html.P('Content for column 1...', className='card-text')
                ])
            ),
        ),
        # Column 2
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 2', className='card-title'),
                    html.P('Content for column 2...', className='card-text')
                ])
            ),
        ),
        # Column 3
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 3', className='card-title'),
                    html.P('Content for column 3...', className='card-text')
                ])
            ),
        ),
        # Column 4
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 4', className='card-title'),
                    html.P('Content for column 4...', className='card-text')
                ])
            ),
        ),
        # Column 5
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 5', className='card-title'),
                    html.P('Content for column 5...', className='card-text')
                ])
            ),
        ),
    ], style={'margin-top': '20px'}),

    # Row with 6 columns
    dbc.Row([
        # Column 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 1', className='card-title'),
                    html.P('Content for column 1...', className='card-text')
                ])
            ),
            width=2  # 1/6 of the width
        ),
        # Column 2
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 2', className='card-title'),
                    html.P('Content for column 2...', className='card-text')
                ])
            ),
            width=2  # 1/6 of the width
        ),
        # Column 3
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 3', className='card-title'),
                    html.P('Content for column 3...', className='card-text')
                ])
            ),
            width=2  # 1/6 of the width
        ),
        # Column 4
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 4', className='card-title'),
                    html.P('Content for column 4...', className='card-text')
                ])
            ),
            width=2  # 1/6 of the width
        ),
        # Column 5
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 5', className='card-title'),
                    html.P('Content for column 5...', className='card-text')
                ])
            ),
            width=2  # 1/6 of the width
        ),
        # Column 6
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 6', className='card-title'),
                    html.P('Content for column 6...', className='card-text')
                ])
            ),
            width=2  # 1/6 of the width
        ),
    ], style={'margin-top': '20px'}),

], fluid=True)


# Update SUBSETOR options based on SETOR
@app.callback(
    Output('subsetor-dropdown', 'options'),
    Input('setor-dropdown', 'value')
)
def update_subsetor_options(selected_setor):
    try:  # Add error handling
        if selected_setor:
            mask = df_fund['SETOR'] == selected_setor
            options = [{'label': i, 'value': i} for i in df_fund[mask]['SUBSETOR'].unique()]
        else:
            options = []
        return options
    except Exception as e:
        print(f"Error: {str(e)}")  # Log error
        return []  # Return empty options in case of error


# Update SEGMENTO options based on SUBSETOR
@app.callback(
    Output('segmento-dropdown', 'options'),
    Input('subsetor-dropdown', 'value')
)
def update_segmento_options(selected_subsetor):
    try:
        if selected_subsetor:
            mask = df_fund['SUBSETOR'] == selected_subsetor
            options = [{'label': i, 'value': i} for i in df_fund[mask]['SEGMENTO'].unique()]
        else:
            options = []
        return options
    except Exception as e:
        print(f"Error: {str(e)}")
        return []


# Update company table based on SETOR, SUBSETOR, and SEGMENTO
@app.callback(
    Output('company-table', 'data'),
    [
        Input('setor-dropdown', 'value'),
        Input('subsetor-dropdown', 'value'),
        Input('segmento-dropdown', 'value'),
    ]
)
def update_table(selected_setor, selected_subsetor, selected_segmento):
    try:
        mask = ((df_fund['SETOR'] == selected_setor) if selected_setor else True) & \
               ((df_fund['SUBSETOR'] == selected_subsetor) if selected_subsetor else True) & \
               ((df_fund['SEGMENTO'] == selected_segmento) if selected_segmento else True)
        data = df_fund[mask][["PREGAO", "TICKER"]].to_dict('records')  # Only show specified columns
        return data
    except Exception as e:
        print(f"Error: {str(e)}")
        return []


# Run the app
app.run_server(mode='external')


In [ ]:
# Function to create the fundamental analysis card
def fundamental_analysis_card():
    return dbc.Card(
        dbc.CardBody([
            dbc.Row([ 
                dbc.Col(html.H2('Análise Fundamentalista'), width=8),
                dbc.Col(html.I(className='fa fa-balance-scale', style={'font-size': '300%'}), width=4, align='center'),
            ], style={'margin-top': '10px'}),
            dbc.Row([
                dbc.Col([
                    ThemeSwitchAIO(aio_id='theme', themes=["url_theme_1", "url_theme_2"]), 
                ]),
            ], style={'margin-top': '10px'}),
            dbc.Row([
                dbc.Col([
                    dbc.Button(f'MAIS INFO SOBRE', href=f"#", target='_blank'), 
                ]),
            ], style={'margin-top': '10px'}),
        ])
    )


In [ ]:
# Function to create the setor and subsetor dropdown selectors
def selectors():
    return dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='setor-dropdown',
                options=[{'label': i, 'value': i} for i in df_fund['SETOR'].unique()],
                value=None,  
                multi=False,
                placeholder="Select a SETOR"
            ),
        ]),
        dbc.Col([
            dcc.Dropdown(
                id='subsetor-dropdown',
                multi=False,
                placeholder="Select a SUBSETOR"
            ),
        ]),
        dbc.Col([
            dcc.Dropdown(
                id='segmento-dropdown',
                multi=False,
                placeholder="Select a SEGMENTO"
            ),
        ]),
    ])


In [ ]:
def content_creator(content_dict):
    """
    Create a list of Dash HTML components.

    Parameters:
        content_dict (dict): A dictionary containing 'title' and 'content' keys.
        'content' can be a string or a list of strings, each of which is rendered as a new paragraph.
    
    Returns:
        list: A list containing Dash HTML components.
    """
    # Ensure content is a list using a ternary expression
    content = content_dict['content'] if isinstance(content_dict['content'], list) else [content_dict['content']]
    
    # Create a paragraph for each item in content
    paragraphs = [html.P(item, className='card-text') for item in content]
    
    # Concatenate title and paragraphs and return
    return [html.H5(content_dict['title'], className='card-title')] + paragraphs


In [ ]:
def row_2_left(col1_content, col2_content):
    return dbc.Row([
        dbc.Col(
            dbc.Card(
                dbc.CardBody(col1_content)
            ),
            width=3
        ),
        dbc.Col(
            dbc.Card(
                dbc.CardBody(col2_content)
            ),
            width=9
        ),
    ], style={'margin-top': '20px'})



In [ ]:
# Function to create the company information card
def company_info_card(company, df, setor, cias_setor, subsetor, cias_subsetor, segmento, cias_segmento):
    return dbc.Card(
        dbc.CardBody([
            dbc.Row(html.H2(f'{company} {df["LISTAGEM"].iloc[0]}')),
            dbc.Row([
                html.P([
                    f"{df['ATIVIDADE'].iloc[0]}",
                ]), 
                html.P([
                    f"{df['ESCRITURADOR'].iloc[0]}", " / ", 
                    html.A(f"{df['CNPJ'].iloc[0]}", href=f"https://www.google.com/search?q={df['CNPJ'].iloc[0]}", target='_blank'), 
                ]), 
            ]),
            dbc.Row([
                dbc.Col([
                    html.Span('Setor'), html.Br(),
                    html.Strong(f'{setor}'), html.Br(),
                    html.P(f'{", ".join(cias_setor)}')
                ]),
                dbc.Col([
                    html.Span('SubSetor'), html.Br(),
                    html.Strong(f'{subsetor}'), html.Br(),
                    html.P(f'{", ".join(cias_subsetor)}')
                ]),
                dbc.Col([
                    html.Span('Segmento'), html.Br(),
                    html.Strong(f'{segmento}'), html.Br(),
                    html.P(f'{", ".join(cias_segmento)}')
                ]),
            ]),
        ])
    )


In [ ]:
# Example usage:
content_1_1 = {
    'title': 'Análise Fundamentalista',
    'content': [
        'Dark/Light Theme', 
        'Select 1', 
        'Select 2', 
        'Select 3', 
        ]
}

content_1_2 = {
    'title': f'{company}',
    'content': [
    f"{df['ATIVIDADE'].iloc[0]}", 
    f"{setor} > {subsetor} > {segmento}", 

    ]
}


In [ ]:
def card(content):
    # Check if header exists and create CardHeader if it does
    header = dbc.CardHeader(content['header']) if 'header' in content else None
    
    # Check if body exists and create CardBody if it does
    body = dbc.CardBody(content['body']) if 'body' in content else None
    
    # Check if footer exists and create CardFooter if it does
    footer = dbc.CardFooter(content['footer']) if 'footer' in content else None
    
    # Return dbc.Card with available components
    return dbc.Card([component for component in [header, body, footer] if component is not None])


In [ ]:
example = {
    'header': [
        html.H2('Header', className='card-title'),
    ], 
    'body': [
        html.H5('Subheader', className='card-title'),
        html.P('Some paragraph text'),
        html.P('Another paragraph'),
        dbc.Button('Click me', color='primary'),
    ], 
    'footer': [
        html.A('Go to Google', href='https://www.google.com', target='_blank'), 
    ], 
}
example_2 = {
    # 'header': [
    #     html.H2('Header', className='card-title'),
    # ], 
    'body': [
        html.H5('Subheader', className='card-title'),
        html.P('Some paragraph text'),
        html.P('Another paragraph'),
        dbc.Button('Click me', color='primary'),
    ], 
    # 'footer': [
    #     html.A('Go to Google', href='https://www.google.com', target='_blank'), 
    # ], 
}

In [ ]:
# App layout
app.layout = dbc.Container([
    card(example), 
    card(example), 
], fluid=True)

# Run the app
app.run_server(mode='external')


In [ ]:
# App layout
app2.layout = dbc.Container([
    # Example Row 1 with 1 column
    dbc.Row([
        dbc.Col(card(example), width=12)
    ], style={'margin-top': '20px'}),
    
    # Example Row 2 with 2 columns
    dbc.Row([
        dbc.Col(card(example), width=6),
        dbc.Col(card(example_2), width=6),
    ], style={'margin-top': '20px'}),
    
    # Example Row 3 with 3 columns
    dbc.Row([
        dbc.Col(card(example), width=4),
        dbc.Col(card(example), width=4),
        dbc.Col(card(example_2), width=4),
    ], style={'margin-top': '20px'}),
    
    # Example Row 4 with 4 columns
    dbc.Row([
        dbc.Col(card(example), width=3),
        dbc.Col(card(example), width=3),
        dbc.Col(card(example), width=3),
        dbc.Col(card(example_2), width=3),
    ], style={'margin-top': '20px'}),
    
    # Example Row 5 with 5 columns
    dbc.Row([
        dbc.Col(card(example),),
        dbc.Col(card(example),),
        dbc.Col(card(example),),
        dbc.Col(card(example),),
        dbc.Col(card(example_2),),
    ], style={'margin-top': '20px'}),

    # Example Row 6 with 6 columns
    dbc.Row([
        dbc.Col(card(example), width=2),
        dbc.Col(card(example), width=2),
        dbc.Col(card(example), width=2),
        dbc.Col(card(example), width=2),
        dbc.Col(card(example), width=2),
        dbc.Col(card(example_2), width=2),
    ], style={'margin-top': '20px'}),
], fluid=True)

# Run the app
app2.run_server(mode='external')


In [ ]:
app.layout = dbc.Container([
   # Row with 1 column
    dbc.Row([
        # Column 1
        dbc.Col(
            card(example), 
            width=12  # Full width
        ),
    ], style={'margin-top': '20px'}),




    # Row with 1 column
    dbc.Row([
        # Column 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 1', className='card-title'),
                    html.P('Content for column 1...', className='card-text')
                ])
            ),
            width=12  # Full width
        ),
    ], style={'margin-top': '20px'}),

    # Row with 2 columns
    dbc.Row([
        # Column 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 1', className='card-title'),
                    html.P('Content for column 1...', className='card-text')
                ])
            ),
            width=6  # Half of the width
        ),
        # Column 2
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 2', className='card-title'),
                    html.P('Content for column 2...', className='card-text')
                ])
            ),
            width=6  # Half of the width
        ),
    ], style={'margin-top': '20px'}),

    # Row with 3 columns
    dbc.Row([
        # Row 2, Col 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 1', className='card-title'),
                    html.P('Content for column 1...', className='card-text')
                ])
            ),
            width=4  # 1/3 of the width
        ),
        # Row 2, Col 2
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 2', className='card-title'),
                    html.P('Content for column 2...', className='card-text')
                ])
            ),
            width=4  # 1/3 of the width
        ),
        # Row 2, Col 3
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 3', className='card-title'),
                    html.P('Content for column 3...', className='card-text')
                ])
            ),
            width=4  # 1/3 of the width
        ),
    ], style={'margin-top': '20px'}),  # Added margin for better visual separation of the rows

    # Row with 4 columns
    dbc.Row([
        # Column 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 1', className='card-title'),
                    html.P('Content for column 1...', className='card-text')
                ])
            ),
            width=3  # 1/4 of the width
        ),
        # Column 2
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 2', className='card-title'),
                    html.P('Content for column 2...', className='card-text')
                ])
            ),
            width=3  # 1/4 of the width
        ),
        # Column 3
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 3', className='card-title'),
                    html.P('Content for column 3...', className='card-text')
                ])
            ),
            width=3  # 1/4 of the width
        ),
        # Column 4
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 4', className='card-title'),
                    html.P('Content for column 4...', className='card-text')
                ])
            ),
            width=3  # 1/4 of the width
        ),
    ], style={'margin-top': '20px'}),

    # Row with 5 columns
    dbc.Row([
        # Column 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 1', className='card-title'),
                    html.P('Content for column 1...', className='card-text')
                ])
            ),
        ),
        # Column 2
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 2', className='card-title'),
                    html.P('Content for column 2...', className='card-text')
                ])
            ),
        ),
        # Column 3
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 3', className='card-title'),
                    html.P('Content for column 3...', className='card-text')
                ])
            ),
        ),
        # Column 4
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 4', className='card-title'),
                    html.P('Content for column 4...', className='card-text')
                ])
            ),
        ),
        # Column 5
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 5', className='card-title'),
                    html.P('Content for column 5...', className='card-text')
                ])
            ),
        ),
    ], style={'margin-top': '20px'}),

    # Row with 6 columns
    dbc.Row([
        # Column 1
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 1', className='card-title'),
                    html.P('Content for column 1...', className='card-text')
                ])
            ),
            width=2  # 1/6 of the width
        ),
        # Column 2
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 2', className='card-title'),
                    html.P('Content for column 2...', className='card-text')
                ])
            ),
            width=2  # 1/6 of the width
        ),
        # Column 3
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 3', className='card-title'),
                    html.P('Content for column 3...', className='card-text')
                ])
            ),
            width=2  # 1/6 of the width
        ),
        # Column 4
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 4', className='card-title'),
                    html.P('Content for column 4...', className='card-text')
                ])
            ),
            width=2  # 1/6 of the width
        ),
        # Column 5
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 5', className='card-title'),
                    html.P('Content for column 5...', className='card-text')
                ])
            ),
            width=2  # 1/6 of the width
        ),
        # Column 6
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H5('Title Col 6', className='card-title'),
                    html.P('Content for column 6...', className='card-text')
                ])
            ),
            width=2  # 1/6 of the width
        ),
    ], style={'margin-top': '20px'}),

], fluid=True)



## Playgroung

In [177]:
import credentials.keys
import requests
from lxml import html
import re
import plotly.express as px
import random


In [120]:
def get_info(response):
    # Parse the HTML snippet
    tree = html.fromstring(response.text)
    
    # Dictionary to store the extracted information
    info = {
        "CNPJ": "",
        "Razão Social": "",
        "Nome Fantasia": "",
        "Data da Abertura": "",
        "Porte": "",
        "Natureza Jurídica": "",
        "Opção pelo MEI": "",
        "Opção pelo Simples": "",
        "Capital Social": "",
        "Tipo": "",
        "Situação": "",
        "Logradouro": "",
        "Bairro": "",
        "CEP": "",
        "Município": "",
        "Estado": ""
    }
    
    # Extract information from <p> elements
    p_elements = tree.xpath('//p')
    for p in p_elements:
        text_content = p.text_content()
        
        # Example to extract "CNPJ"
        if 'CNPJ:' in text_content:
            cnpj = re.search(r'\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}', text_content)
            if cnpj:
                info['CNPJ'] = cnpj.group(0)
        
        # Similarly, you can add conditions to extract other pieces of information
        # Example to extract "Razão Social"
        if 'Razão Social:' in text_content:
            razao_social = p.xpath('.//b[@class="copy"]/text()')
            if razao_social:
                info['Razão Social'] = razao_social[0]
        
        if 'Nome Fantasia:' in text_content:
            nome_fantasia = p.xpath('.//b[@class="copy"]/text()')
            if nome_fantasia:
                info['Nome Fantasia'] = nome_fantasia[0]
        
        # Extract "Data da Abertura"
        if 'Data da Abertura:' in text_content:
            data_abertura = re.search(r'\d{2}/\d{2}/\d{4}', text_content)
            if data_abertura:
                info['Data da Abertura'] = data_abertura.group(0)
        
        # Extract "Porte"
        if 'Porte:' in text_content:
            porte = text_content.split(':')[-1].strip()
            info['Porte'] = porte
        
        # Extract "Natureza Jurídica"
        if 'Natureza Jurídica:' in text_content:
            natureza_juridica = p.xpath('.//b[@class="copy"]/text()')
            if natureza_juridica:
                info['Natureza Jurídica'] = natureza_juridica[0]
        
        # Extract "Opção pelo MEI"
        if 'Opção pelo MEI:' in text_content:
            opcao_mei = text_content.split(':')[-1].strip()
            info['Opção pelo MEI'] = opcao_mei
        
        # Extract "Opção pelo Simples"
        if 'Opção pelo Simples:' in text_content:
            opcao_simples = text_content.split(':')[-1].strip()
            info['Opção pelo Simples'] = opcao_simples
        
        # Extract "Capital Social"
        if 'Capital Social:' in text_content:
            capital_social_str = p.xpath('.//b[@class="copy"]/text()')
            if capital_social_str:
                # Extract numbers, ignore R$ and commas, replace decimal separator
                capital_social_num = re.sub(r'[^\d,]', '', capital_social_str[0]).replace(',', '.')
                info['Capital Social'] = float(capital_social_num)
    
            # Extract "Tipo"
        if 'Tipo:' in text_content:
            tipo = p.xpath('.//b[@class="copy"]/text()')
            if tipo:
                info['Tipo'] = tipo[0]
        
        # Extract "Situação"
        if 'Situação:' in text_content:
            situacao = p.xpath('.//b[@class="copy"]/text()')
            if situacao:
                info['Situação'] = situacao[0]

        # Extract "Logradouro"
        if 'Logradouro:' in text_content:
            logradouro = p.xpath('.//b[@class="copy"]/text()')
            if logradouro:
                info['Logradouro'] = logradouro[0]

        # Extract "Bairro"
        if 'Bairro:' in text_content:
            bairro = p.xpath('.//b[@class="copy"]/text()')
            if bairro:
                info['Bairro'] = bairro[0]

        # Extract "CEP"
        if 'CEP:' in text_content:
            cep = re.search(r'\d{5}-\d{3}', text_content)
            if cep:
                info['CEP'] = cep.group(0)
        
        # Extract "Município"
        if 'Município:' in text_content:
            municipio = text_content.split(':')[-1].strip()
            info['Município'] = municipio
        
        # Extract "Estado"
        if 'Estado:' in text_content:
            estado = text_content.split(':')[-1].strip()
            info['Estado'] = estado

    # Return the extracted information
    return info


In [124]:
def get_branches(response):
    """
    Extract and clean CNPJ numbers from an HTTP response object.

    Args:
    - response: requests.Response, HTTP response containing the CNPJ data.

    Returns:
    - list of str, Cleaned CNPJ numbers.
    """
    xpath = '/html/body/div[1]/div/div/div[2]/div[1]'

    tree = html.fromstring(response.content)
    data = tree.xpath(xpath)
    # Check if data was found
    branches = []
    if data:
        branches = [el.text_content().strip() for el in data[0].xpath('.//a')]

        # get only CNPJ info
        cnpj_pattern = re.compile(r'\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}')
        
        # Extract CNPJs using the regex pattern
        branches = [re.search(cnpj_pattern, item).group(0) for item in branches if re.search(cnpj_pattern, item)]
        
        # Remove punctuation from CNPJs
        branches = [''.join(re.findall(r'\d', cnpj)) for cnpj in branches]
        branches = list(set(branches))
        branches = sorted(branches)

    return branches


In [186]:
api_key=credentials.keys.google_geocode_maps

company_list = ['47960950000121']

In [194]:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 5.1; rv:36.0) Gecko/20100101 Firefox/36.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/83.0.478.37",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 13_5_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPad; CPU OS 13_5_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.4 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:57.0) Gecko/20100101 Firefox/57.0",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/53.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134"
]


In [203]:
start_time = run.time.time()
for c, cnpj in enumerate(company_list):
    company = {}
    url = f'https://cnpj.biz/{cnpj}'

    # Define headers to mimic a request from a browser
    headers = {
        "User-Agent": random.choice(USER_AGENTS)
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        company[cnpj] = {}
        branches = get_branches(response)
        start_time_2 = run.time.time()
        for i, branch in enumerate(branches):
            url = f'https://cnpj.biz/{branch}'
            # Define headers to mimic a request from a browser
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
            }
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                company[cnpj][branch] = get_info(response)
                print(branch, run.remaining_time(start_time_2, len(branches), i))
            else:
                print(f'{branch} erro {response.status_code}')
            # Sleep for 5 seconds every 50 companies
            if i % 50 == 0 and i > 0:
                print('...preventive waiting')
                run.time.sleep(5)
        print(cnpj, run.remaining_time(start_time, len(company_list), c))
    else:
        print(f'{cnpj} erro {response.status_code}')

In [196]:
df = pd.DataFrame(filiais).T

In [198]:
df[df['CNPJ'] != '']

,CNPJ,Razão Social,Nome Fantasia,Data da Abertura,Porte,Natureza Jurídica,Opção pelo MEI,Opção pelo Simples,Capital Social,Tipo,Situação,Logradouro,Bairro,CEP,Município,Estado
47956099000163,47.956.099/0001-63,Confeccoes Rajik LTDA,,28/11/1981,Micro Empresa,Sociedade Empresária Limitada,Não,Não,400000.0,Matriz,Baixada,,,,Campinas,São Paulo
47958327000134,47.958.327/0001-34,Lamar Ind e Com de Moveis Para Escritorio LTDA,,01/12/1981,Micro Empresa,Sociedade Empresária Limitada,Não,Não,10000.0,Matriz,Inapta,,,,São Paulo,São Paulo
47958855000193,47.958.855/0001-93,Msm-Produtos Para Calcados LTDA,,28/03/1966,Sem Enquadramento,Sociedade Empresária Limitada,Não,Não,14694778.5,Matriz,Ativa,"Avenida Rio Branco, 520",Estacao,14405-080,Franca,São Paulo
47959697000196,47.959.697/0001-96,Amazonas Industria e Comercio LTDA,Amazonas,30/03/1966,Sem Enquadramento,Sociedade Empresária Limitada,Não,Não,145526355.0,Matriz,Ativa,"Avenida Rio Branco 745, 745",Vila Santos Dumont,14405-901,Franca,São Paulo
47960950000202,47.960.950/0002-02,Magazine Luiza S/A,Magazine Luiza,18/08/1983,Sem Enquadramento,Sociedade Anônima Aberta,Não,Não,12552162483.75,Filial,Ativa,"Avenida Sao Carlos, 1428",Centro,13560-011,São Carlos,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47960950113163,47.960.950/1131-63,Magazine Luiza S/A,Magazine Luiza S/A,08/08/2018,Sem Enquadramento,Sociedade Anônima Aberta,Não,Não,12552162483.75,Filial,Ativa,"Rua Sete de Setembro, 345",Centro,13490-000,Cordeirópolis,São Paulo
47960950113406,47.960.950/1134-06,Magazine Luiza S/A,Magazine Luiza S/A,21/08/2018,Sem Enquadramento,Sociedade Anônima Aberta,Não,Não,12552162483.75,Filial,Ativa,"Rua Armando Steck, 408",Jardim Niero I,13290-000,Louveira,São Paulo
47960950113597,47.960.950/1135-97,Magazine Luiza S/A,Magazine Luiza S/A,21/08/2018,Sem Enquadramento,Sociedade Anônima Aberta,Não,Não,12552162483.75,Filial,Ativa,"Rua Prudente de Moraes, 409",Centro,13390-000,Rio das Pedras,São Paulo
47960950113830,47.960.950/1138-30,Magazine Luiza S/A,Magazine Luiza S/A,28/08/2018,Sem Enquadramento,Sociedade Anônima Aberta,Não,Não,12552162483.75,Filial,Ativa,"Praca Presidente Kennedy, 51A",Centro,18520-000,Cerquilho,São Paulo


403

In [174]:
t = get_info(response)
t

{'CNPJ': '',
 'Razão Social': '',
 'Nome Fantasia': '',
 'Data da Abertura': '',
 'Porte': '',
 'Natureza Jurídica': '',
 'Opção pelo MEI': '',
 'Opção pelo Simples': '',
 'Capital Social': '',
 'Tipo': '',
 'Situação': '',
 'Logradouro': '',
 'Bairro': '',
 'CEP': '',
 'Município': '',
 'Estado': ''}

In [148]:
df = pd.DataFrame(company['47960950000121']).T
df

,CNPJ,Razão Social,Nome Fantasia,Data da Abertura,Porte,Natureza Jurídica,Opção pelo MEI,Opção pelo Simples,Capital Social,Tipo,Situação,Logradouro,Bairro,CEP,Município,Estado
47956099000163,47.956.099/0001-63,Confeccoes Rajik LTDA,,28/11/1981,Micro Empresa,Sociedade Empresária Limitada,Não,Não,400000.0,Matriz,Baixada,,,,Campinas,São Paulo
47958327000134,47.958.327/0001-34,Lamar Ind e Com de Moveis Para Escritorio LTDA,,01/12/1981,Micro Empresa,Sociedade Empresária Limitada,Não,Não,10000.0,Matriz,Inapta,,,,São Paulo,São Paulo
47958855000193,47.958.855/0001-93,Msm-Produtos Para Calcados LTDA,,28/03/1966,Sem Enquadramento,Sociedade Empresária Limitada,Não,Não,14694778.5,Matriz,Ativa,"Avenida Rio Branco, 520",Estacao,14405-080,Franca,São Paulo
47959697000196,47.959.697/0001-96,Amazonas Industria e Comercio LTDA,Amazonas,30/03/1966,Sem Enquadramento,Sociedade Empresária Limitada,Não,Não,145526355.0,Matriz,Ativa,"Avenida Rio Branco 745, 745",Vila Santos Dumont,14405-901,Franca,São Paulo
47960950000202,47.960.950/0002-02,Magazine Luiza S/A,Magazine Luiza,18/08/1983,Sem Enquadramento,Sociedade Anônima Aberta,Não,Não,12552162483.75,Filial,Ativa,"Avenida Sao Carlos, 1428",Centro,13560-011,São Carlos,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47960950190230,,,,,,,,,,,,,,,,
47960950190311,,,,,,,,,,,,,,,,
47960950190583,,,,,,,,,,,,,,,,
47960950190664,,,,,,,,,,,,,,,,


In [61]:
fig = px.scatter_mapbox(baixadas, lat='Latitude', lon='Longitude', 
                        hover_name='Branch_Name', 
                        hover_data=['Status', 'Street', 'City', 'State', 'CNPJ'],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()